In [8]:
import urllib.request
url = 'http://hustle-sa.ru/rating/dancers.zip'
filename = 'dancers.zip'
urllib.request.urlretrieve(url, filename)

('dancers.zip', <http.client.HTTPMessage at 0x1785edc6760>)

In [9]:
import zipfile
archive = zipfile.ZipFile('dancers.zip', 'r')
archive.extractall('.')
archive.close()


In [2]:
#Блок используемых библиотек
#import wget
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import warnings


warnings.filterwarnings('ignore')
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 150
pd.set_option('display.float_format', '{:.2f}'.format)

In [452]:
clubs=pd.read_excel(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\01.01.2023\dancers.xlsm', sheet_name='клубы')
dancers=pd.read_excel(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\dancers.xlsm', sheet_name='список танцоров')
events_classic=pd.read_excel(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\dancers.xlsm', sheet_name='rating')
events_jnj=pd.read_excel(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\dancers.xlsm', sheet_name='rating ДнД')
events_classic_res = pd.read_excel(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\dancers.xlsm', sheet_name='rating')
events_jnj_res = pd.read_excel(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\dancers.xlsm', sheet_name='rating ДнД')

## Формируем информацию о клубах

In [453]:
clubs.head()

,№п.п.,Клуб,Руководитель/Контактное лицо,Сайт1,Сайт2,Старые названия,Комментарий
0,1,2H-club,NaN,NaN,NaN,NaN,NaN
1,2,AT-Dance,NaN,NaN,NaN,NaN,NaN
2,3,Baila&Dance (г.Красноярск),NaN,NaN,NaN,NaN,NaN
3,4,Caminito (г.СПб),NaN,NaN,NaN,NaN,"нет хастла, танго"
4,5,Dance College Club,NaN,NaN,NaN,NaN,NaN


In [454]:
clubs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281 entries, 0 to 280
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   №п.п.                         281 non-null    object
 1   Клуб                          281 non-null    object
 2   Руководитель/Контактное лицо  70 non-null     object
 3   Сайт1                         86 non-null     object
 4   Сайт2                         8 non-null      object
 5   Старые названия               10 non-null     object
 6   Комментарий                   19 non-null     object
dtypes: object(7)
memory usage: 15.5+ KB


In [455]:
clubs.columns = ['club_id', 'club_name', 'contact', 'website_1', 'website_2', 'old_name', 'comments']
clubs = clubs[['club_id', 'club_name']] #оставим только регулярную и полную информацию

**Комментарий**:
- система присвоения id клубам менялась несколько раз, первоначально номера присваивались последовательно, начиная с 27.06.2010, клубу присваивался номер по дате внесения сведения о клубе в базу в формате ггммдд или ггммдд_1-2 (видимо в тех случаях, когда в один день в базу вносились сведения о нескольких клубах, начиная с 16.02.2019 номер присваивался по дате в формате ггггммдд)

In [456]:
clubs.club_id = clubs.club_id.replace("_", "", regex=True)

In [457]:
#добавим потерянные клубы в базу, присвоив им id в соответствии с датой их первого появления в базе
clubs_ap=pd.DataFrame({'club_id':['170729', '190104', '191018', '20210328', '20211030'],
                          'club_name':['самост. (г. Южно-Сахалинск)','UTSC Choice (г.Graz)', 'Вместе (г.Харьков)', 'Comfort Dance (г.Иваново)', 'самост. (г.Новосибирск)']
                        })
clubs_ap

,club_id,club_name
0,170729,самост. (г. Южно-Сахалинск)
1,190104,UTSC Choice (г.Graz)
2,191018,Вместе (г.Харьков)
3,20210328,Comfort Dance (г.Иваново)
4,20211030,самост. (г.Новосибирск)


In [458]:
#добавим эти клуюы к остальным
clubs = clubs.append(clubs_ap, ignore_index = True)
clubs

,club_id,club_name
0,1,2H-club
1,2,AT-Dance
2,3,Baila&Dance (г.Красноярск)
3,4,Caminito (г.СПб)
4,5,Dance College Club
...,...,...
281,170729,самост. (г. Южно-Сахалинск)
282,190104,UTSC Choice (г.Graz)
283,191018,Вместе (г.Харьков)
284,20210328,Comfort Dance (г.Иваново)


In [459]:
#В дальнейшем нам понадобятся сведения об id клубов в привязке к их оригинальному названию в базе, поэтому создадим дубликат этих сведений
clubs_1 = clubs.append(clubs_ap, ignore_index = True)
clubs_1 = clubs_1.drop_duplicates()

**Комментарий**:
главным недостатком этих данных является то, что наименование города, где находится клуб, указывается вместе с наименование клуба, также у нас есть клубы без населенного пункта, как правило, по умолчанию в базе так учитывались клубы из Москвы, но это необходимо будет дополнительно проверить.

In [460]:
#отделим город от наименования клуба по открывающейся скобке
club_city = clubs['club_name'].str.split('(',expand=True)
club_city.columns = ['club_name', 'club_city']

#очистим значения в столбце club_city от символов 'г.' и ')'
cols = ['club_city']
for col in cols:
    club_city[col] = club_city[col].map(lambda x: str(x).lstrip('г.').rstrip(')'))

#Приведем названия городов к единообразию
club_city['club_city'] = club_city['club_city'].replace({
                                                       'СПб' : 'Санкт-Петербург',
                                                       'В.Новгород' : 'Великий Новгород',
                                                       'Моск.обл.' : 'Московская область',
                                                       'Lausanne' : 'Лозанна',
                                                       'Poznan' : 'Познань',
                                                       'Graz': 'Грац',
                                                       'Glasgow' : 'Глазго',
                                                       ' Южно-Сахалинск' : 'Южно-Сахалинск'
                                                                                               
}, regex=True)
club_city.loc[club_city['club_city'] == 'Сахалин', 'club_city'] = 'Южно-Сахалинск'
# в исходном файле в комментарии указано, что старое наименование этого клуба Let's Go (г.Красногорск), 
# поэтому логично указать этот город в наших данных

club_city['club_city'] = club_city['club_city'].str.replace(
                                                       'Московская область', 'Красногорск',
                                                        regex=True
)

club_city.columns = ['club_name', 'club_city']

#очистим значения в столбце club_city от символов 'г.' и ')'
cols = ['club_city']
for col in cols:
    club_city[col] = club_city[col].map(lambda x: str(x).lstrip('г.').rstrip(')'))

#Приведем названия городов к единообразию
club_city['club_city'] = club_city['club_city'].replace({
                                                       'СПб' : 'Санкт-Петербург',
                                                       'В.Новгород' : 'Великий Новгород',
                                                       'Моск.обл.' : 'Московская область',
                                                       'Lausanne' : 'Лозанна',
                                                       'Poznan' : 'Познань',
                                                       'Graz': 'Грац',
                                                       'Glasgow' : 'Глазго',
                                                       ' Южно-Сахалинск' : 'Южно-Сахалинск'
                                                        
                                                      
}, regex=True)
club_city.loc[club_city['club_city'] == 'Сахалин', 'club_city'] = 'Южно-Сахалинск'

# в исходном файле в комментарии указано, что старое наименование этого клуба Let's Go (г.Красногорск), 
# поэтому логично указать этот город в наших данных

club_city['club_city'] = club_city['club_city'].str.replace(
                                                       'Московская область', 'Красногорск',
                                                        regex=True
)

In [461]:
# теперь посмотрим названия клубов, у которых город указан не был
club_city[club_city['club_city'] == 'None']['club_name'].unique()

array(['2H-club', 'AT-Dance', 'Dance College Club', 'Nice Dance',
       'DanceStyle', 'F Club', 'Hostel Hustle', 'Ivara', 'N-Club',
       'Nova Dance Studio', 'Open Dance Studio', 'Profi-Dance', 'RedHop',
       'Reverie Dance', 'S-клуб', 'Tanto', 'Vesta', 'Sergio dance',
       'Алекс Хастл-Клуб', 'Альтернатива', 'Априори', 'БиС', 'Богема',
       'В-12', 'Визави', 'Виктория', 'Данцарто', 'Движение', 'Дизайн',
       'Индиго', 'Красный уголок', 'Остров', 'Профи-Данс', 'Пульс',
       'Ритм', 'Самарский хастл-клуб', 'Спартак', 'Стиль',
       'студия Александра Красильникова', 'Студия Виктории Деминой',
       'Студия С.Спиридонова', 'СХК', 'ТВИКS', 'Динамика', 'Хастл-Центр',
       'Центральный', 'Экспромт', 'Орион', 'Smile', 'LaDanza', 'AltroOne',
       'Германия', 'DanceFirst', 'Magma', 'Мегаполис', 'Dance Drive',
       'Эксельсиор', 'YouDance', 'Walzer', 'Poland', 'Aizek`s',
       'DanceLab', 'Daily dance', 'AlphaDance', 'самост.',
       'Мастерская Свободного Танца', 'Мартэ'

**Комментарий**:
- очень много названий клубов, которые сложно назвать широко известными, вероятно это связано с начальным периодом образования Ассоциации
- если принимать во внимание, что по умолчанию отсутствие города может трактоваться, как то что клуб находится в Москве, вызывают следующие клубы:
   - Самарский хастл-клуб (вероятно клуб из Самары)
   - Германия (скорее всего так были отмечены какие то иностранные танцоры из Германии)
   - Poland (вероятно это какой то польский клуб)
   - Триумф плюс (клуб из Санкт-Петербурга)
- по остальным клубам, вероятно необходимо провести проверку по танцорам, которые приписаны к этому клубу в базе или осуществить поиск в интернете

In [462]:
club_city.loc[club_city['club_name'] == 'Самарский хастл-клуб', 'club_city'] = 'Самара'
club_city.loc[club_city['club_city'] == 'Чебоксары', 'club_name'] = 'Ритм'
club_city.loc[club_city['club_name'] == 'Германия', 'club_city'] = 'Гамбург'
club_city['club_name'] = club_city['club_name'].str.replace(
                                                       'Германия', 'Mein Tanzstudio',
                                                        regex=True
)
club_city.loc[club_city['club_name'] == 'Poland', 'club_city'] = 'Краков'
club_city['club_name'] = club_city['club_name'].str.replace(
                                                       'Poland', 'Akademia Tańca',
                                                        regex=True
)
club_city.loc[club_city['club_name'] == 'Триумф плюс', 'club_city'] = 'Санкт-Петербург'
club_city['club_city'] = club_city['club_city'].str.replace(
                                                       'None', 'Москва'
                                                      
)
#очистим значения в столбце club_name от пробела в конце названия клуба
cols = ['club_name']
for col in cols:
    club_city[col] = club_city[col].map(lambda x: str(x).rstrip(' '))

#заменим обработанные сведения о названиях клубов в исходном файле
clubs['club_name'] = club_city['club_name']
#добавим сведения о городе
clubs['club_city'] = club_city['club_city']
#Заменим 'самост.' на 'самостоятельно'
clubs.loc[clubs['club_name'] == 'самост.', 'club_name'] = 'самостоятельно'
#Напишем формулу, которая добавит в наши данные страну клуба по городу
def get_country(city):
    if city == 'Минск':
        return 'Беларусь'
    elif city in ('Краков', 'Познань'):
        return 'Польша'
    elif city in ('Павлодар', 'Караганда'):
        return 'Казахстан'
    elif city in ('Харьков', 'Киев'):
        return 'Украина'
    elif city == 'Лозанна':
        return 'Швейцария'
    elif city in ('Дортмунд', 'Гамбург'):
        return 'Германия'
    elif city == 'Глазго':
        return 'Шотландия'
    elif city == 'Грац':
        return 'Австрия'    
    else:
        return 'Россия'
clubs['club_country'] = clubs['club_city'].apply(get_country)

In [463]:
clubs

,club_id,club_name,club_city,club_country
0,1,2H-club,Москва,Россия
1,2,AT-Dance,Москва,Россия
2,3,Baila&Dance,Красноярск,Россия
3,4,Caminito,Санкт-Петербург,Россия
4,5,Dance College Club,Москва,Россия
...,...,...,...,...
281,170729,самостоятельно,Южно-Сахалинск,Россия
282,190104,UTSC Choice,Грац,Австрия
283,191018,Вместе,Харьков,Украина
284,20210328,Comfort Dance,Иваново,Россия


**Комментарий**:
- на основании источника сформированы сведения об индивидуальном номере клуба, его названии, городе и стране, где он находится
- скорректированы выявленные неточности

## Формируем информацию о танцорах

**Комментарий**:
- необходимо удалить столбцы: 'Unnamed: 2', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'A', 'Ch', '1', а также строку с индексом 0 и пустые строки
- сведения об имени отчестве и фамилии корректно будет разбить на отдельные столбцы
- необходимо подумать о том нужны ли нам сведения о прежних фамилиях
- информацию о клубах необходимо привести в соответствие с наименованиями клубов полученными в предыдущем разделе
- из сведений об источнике необходимо сформировать сведения о дате внесения танцора в базу данных и вероятно о турнире, после которого в базу были внесены эти сведения


In [464]:
#удаляем ненужные столбцы
dancers = dancers.drop(['Unnamed: 2', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'A', 'Ch', 1], axis=1)

#удаляем ненужные строки
dancers = dancers.drop([0], axis=0).reset_index(drop=True)

#корректируем названия столбцов
dancers.columns = ['dancer_id', 'dancer_name', 'dancer_name_old', 'club_name', 'class', 'class_jnj', 'gender', 'source']

#удалим пустые строки
dancers = dancers.dropna(axis=0,how='all')

#удалим артефакты
dancers = dancers.loc[~dancers['dancer_name'].isna()]

**Комментарий**: 
Получим данные о дате внесения танцора в базу данных

In [465]:
dancers_source = dancers[['dancer_id', 'source']]
dancers_source[['source_tmp', 'source_false']] = dancers_source['source'].str.split(' ', 1, expand=True)
dancers_source[['source_false_1', 'source_false_2']] = dancers_source['source_tmp'].str.split('-', 1, expand=True)
dancers_source[['start_date', 'source_false_3']] = dancers_source['source_false_1'].str.split(',', 1, expand=True)
dancers_source = dancers_source.drop(['source_tmp', 'source_false', 'source_false_1', 'source_false_2', 'source_false_3'], axis=1)

#скорректируем данные у танцоров, которые были внесены в базу данных по заявлению
dancers_source.loc[dancers_source['dancer_id'] == 1058, 'start_date'] = '20071028'
dancers_source.loc[dancers_source['dancer_id'] == 1216, 'start_date'] = '20080316'
dancers_source.loc[dancers_source['dancer_id'] == 1217, 'start_date'] = '20080316'
dancers_source.loc[dancers_source['dancer_id'] == 1255, 'start_date'] = '20080427'
dancers_source.loc[dancers_source['dancer_id'] == 1298, 'start_date'] = '20080608'
dancers_source.loc[dancers_source['dancer_id'] == 1299, 'start_date'] = '20080608'
dancers_source.loc[dancers_source['dancer_id'] == 1328, 'start_date'] = '20080720'
dancers_source.loc[dancers_source['dancer_id'] == 1329, 'start_date'] = '20080720'
dancers_source.loc[dancers_source['dancer_id'] == 1330, 'start_date'] = '20080720'
dancers_source.loc[dancers_source['dancer_id'] == 1331, 'start_date'] = '20080720'
dancers_source.loc[dancers_source['dancer_id'] == 1332, 'start_date'] = '20080720'
dancers_source.loc[dancers_source['dancer_id'] == 1333, 'start_date'] = '20080720'
dancers_source.loc[dancers_source['dancer_id'] == 1353, 'start_date'] = '20080824'
dancers_source.loc[dancers_source['dancer_id'] == 1354, 'start_date'] = '20080824'

#скорректируем ошибочный формат даты
dancers_source.loc[dancers_source['start_date'] == '20091402', 'start_date'] = '20090214'
dancers_source.loc[dancers_source['start_date'] == '20211303', 'start_date'] = '20210313'
dancers_source.loc[dancers_source['start_date'] == '20140631', 'start_date'] = '20140531'
dancers_source['start_date'] = pd.to_datetime(dancers_source['start_date'], errors = 'ignore', format='%Y%m%d').dt.strftime('%Y.%m.%d')
dancers['start_date'] = dancers_source['start_date'].astype('datetime64')
dancers['dancer_id'] = dancers['dancer_id'].astype('int')

**Комментарий**:
Сформруем данный о ФИО танцоров

In [466]:
# разделим информацию в series на столбцы last_name, first_name  и middle_name
dancer_name = dancers['dancer_name'].str.split(' ',expand=True)

#скорректируем сложные имена
dancer_name.loc[dancer_name[0] == 'Ал-Фахри', 1] = 'Фарес Файсал'
dancer_name.loc[dancer_name[0] == 'Ал-Фахри', 2] = 'Ахмед Мабхот'
dancer_name.loc[dancer_name[0] == 'Тугаринова', 1] = 'Наталья'
dancer_name.loc[dancer_name[0] == 'Тугаринова', 2] = 'Александровна'
dancer_name.loc[dancer_name[0] == 'Мелла', 1] = 'Хамед Алан'
dancer_name.loc[dancer_name[0] == 'Мелла', 2] = 'Лезгинович'
dancer_name.loc[(dancer_name[0] == 'Баширова')&(dancer_name[1] == 'Фарах'), 2] = 'Вагиф кызы'
dancer_name.loc[(dancer_name[0] == 'Наимов')&(dancer_name[1] == 'Дилшод'), 2] = 'Гайрат Угли'
dancer_name.loc[dancer_name[1] == 'Мария\xa0', 1] = 'Мария'

#удалим столбцы 3 и 4
dancer_name = dancer_name.drop([3, 4], axis=1)

dancer_name.columns = ['last_name', 'first_name', 'middle_name']

#добавим данные в файл с танцорами
dancers = dancers.drop(['dancer_name'], axis=1)
dancers['last_name'] = dancer_name['last_name']
dancers['first_name'] = dancer_name['first_name']
dancers['middle_name'] = dancer_name['middle_name']

In [467]:
#скорректируем расположение столбцов
dancers = dancers[['dancer_id', 'last_name', 'first_name', 'middle_name', 'dancer_name_old', 'club_name', 'class', 'class_jnj', 'gender', 'start_date']]

**Комментарий**:
Информация о прежних фамилиях танцоров нам будет нужна в дальнейшем.
Проблема в том, что в базе данных учитываются только фамилии без указания даты внесения изменений в базу данных, теоретически эту информацию можно получить распарсив форум АСХ, где подаются заявления на внесение изменений в базу данных, но встает вопрос в соответствии пользы и трудозатрат на получение этих данных. Поэтому на данном этапе пока не учитываем эту информацию.

**Комментарий**:
Основная проблема с данным столбцом заключается в том, что танцоры в АСХ могут указывать себе несколько клубов в клубную принадлежность, при этом все клубы в базе указаны в одной ячейке. План работы с яцейкой заключается в том, чтобы разделить все клубы, указанные в клубной принадлежности танцора на отдельные ячейки и в последующем заменить наименования клубов на id клубов из сведений о клубах, которые мы получили ранее.

In [468]:
#Скорректируем некорректное написание наименований клубов
dancers['club_name'] = dancers['club_name'].replace({
                'Красный Уголок,Движение' : 'Красный уголок,Движение',
                'Красный Уголок' : 'Красный уголок',
                'Dancelab' : 'DanceLab',
                'ТвикS' : 'ТВИКS',
                'Хастл-Сити (г.Новокузнецк)' : 'Хастл-сити (г.Новокузнецк)',
                'ДИСКВАЛИФ.бессрочно Ivara' : 'Ivara',
                'Седьмое небо (г.Железногорск)' : 'Седьмое Небо (г.Железногорск)',
                'Хастл-центр' : 'Хастл-Центр',
                'Самост.' : 'самост.',
                'NoN-STOP (г.СПб)' : 'NON-STOP (г.СПб)',
                'Вираж (г.Новосибирcк)' : 'Вираж (г.Новосибирск)',
                'Hustledance (г.Минск)' : 'HustleDance (г.Минск)',
                'MoveOn' : 'MoveOn (г.Lausanne)',
                'Youdance' : 'YouDance',
                'Танцевальная мастерская Мансарда (г.Тверь)' : 'Мансарда (г.Тверь)',
                'Данс-Дизайн (г.Курск)' : 'Дизайн (г.Курск)',
                'Танцевальная Мастерская "Мансарда" (г.Тверь)' : 'Мансарда (г.Тверь)',
                'Dance Café (Минск)' : 'Dance Cafe (г.Минск)',
                'моего клуба нет в базе' : 'cамост. (г. Южно-Сахалинск)',
                'Ipanema (г. Новосибирск)' : 'Ipanema (г.Новосибирск)',
                ' Ivara' : 'Ivara'}, regex=True)

dancers.loc[dancers['club_name'] == 'Хастл-Сити (г.Новокузнецк)', 'club_name'] = 'Хастл-сити (г.Новокузнецк)'
dancers.loc[dancers['club_name'] == 'Седьмое небо (г.Железногорск)', 'club_name'] = 'Седьмое Небо (г.Железногорск)'
dancers.loc[dancers['club_name'] == 'NoN-STOP (г.СПб)', 'club_name'] = 'NON-STOP (г.СПб)'
dancers.loc[dancers['club_name'] == 'Вираж (г.Новосибирcк)', 'club_name'] = 'Вираж (г.Новосибирск)'
dancers.loc[dancers['club_name'] == 'Hustledance (г.Минск)', 'club_name'] = 'HustleDance (г.Минск)'
dancers.loc[dancers['club_name'] == 'Танцевальная мастерская Мансарда (г.Тверь)', 'club_name'] = 'Мансарда (г.Тверь)'
dancers.loc[dancers['club_name'] == 'Данс-Дизайн (г.Курск)', 'club_name'] = 'Дизайн (г.Курск)'
dancers.loc[dancers['club_name'] == 'Танцевальная Мастерская "Мансарда" (г.Тверь)', 'club_name'] = 'Мансарда (г.Тверь)'
dancers.loc[dancers['club_name'] == 'Dance Café (Минск)', 'club_name'] = 'Dance Cafe (г.Минск)'
dancers.loc[dancers['club_name'] == 'Ipanema (г. Новосибирск)', 'club_name'] = 'Ipanema (г.Новосибирск)'
dancers.loc[dancers['club_name'] == 'Magnit', 'club_name'] = 'ШкоLa Magnit (г.СПб)'
dancers.loc[dancers['club_name'] == 'Magnit,Caminito (г.СПб)', 'club_name'] = 'ШкоLa Magnit (г.СПб), Caminito (г.СПб)'
dancers.loc[dancers['club_name'] == 'cамост. (г. Южно-Сахалинск)', 'club_name'] = 'самост. (г. Южно-Сахалинск)'


In [470]:
#разделим данные в столбце по символу `,` и сохраним в отдельную переменную
club = dancers['club_name'].str.split(',',expand=True)

In [472]:
club.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 0 to 11999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       12000 non-null  object
 1   1       747 non-null    object
 2   2       38 non-null     object
dtypes: object(3)
memory usage: 375.0+ KB


**Комментарий**:
Поскольку в данных о танцорах наименования клубов указаны вместе с городом, где они находятся, в разделе про клубы я сделал дополнительный датафрейм `clubs_1` (фактически это промежуточная иттерация). Суть идеи заключается в том, что в данном датафрейме есть наименование клуба с городом и `id` клуба.

In [473]:
#напишем функцию, которая берет название клуба и возвращает его id
def search_id(name):
    for index, row in clubs_1.iterrows():
        if name == row[1]:
            return row[0]

In [474]:
#заменим в нашей временной таблице `club` названия клубов на их id
club = club.applymap(lambda x: search_id(x))

In [475]:
club.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 0 to 11999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       12000 non-null  object 
 1   1       741 non-null    object 
 2   2       38 non-null     float64
dtypes: float64(1), object(2)
memory usage: 375.0+ KB


In [476]:
club = club.fillna(0)

In [477]:
club = club.astype('int64')

**Комментарий**: 
Сформируем сводный файл по всем танцорам и конкурсам, в которых они участвовали

In [478]:
club['dancer_id'] = dancers['dancer_id']
club_dancer = club.set_index('dancer_id')
club_dancer = club_dancer.T
club_dancer = club_dancer.stack().reset_index()
club_dancer = club_dancer[club_dancer[0] != 0]
club_dancer = club_dancer.drop(columns=['level_0'])
club_dancer = club_dancer.reset_index(drop=True)
club_dancer = club_dancer.rename(columns={0 : 'club_id'})
club_dancer = club_dancer.sort_values(by='dancer_id')

In [487]:
club_dancer

,dancer_id,club_id
0,1,176
1,4,35
2,16,16
12000,28,26
3,28,59
...,...,...
12740,12775,81
11996,12776,160323
11997,12777,12
11998,12778,12


In [661]:
#Скорректируем порядок столбцов
dancers = dancers[['dancer_id', 'start_date', 'last_name', 'first_name', 'middle_name', 'class', 'class_jnj', 'gender']]

## Формируем информацию о турнирах

### Информация о турнирах classic и hybrid

**Комментарий**:
Удаление лишних строк и столбцов

In [491]:
events_classic.drop(events_classic.columns[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]], axis=1, inplace=True)
events_classic.columns=events_classic.loc[0]
events_classic = events_classic.iloc[1:9]
events_classic = events_classic.T
events_classic = events_classic.reset_index(drop=False)
events_classic['event_info'] = events_classic[[2, 3, 4, 5, 6, 7, 8]].astype('str').agg(','.join, axis=1) 
events_classic.drop(events_classic.columns[[2, 3, 4, 5, 6, 7, 8]], axis=1, inplace=True)

**Комментарий**:
получение и стандартизация информации о номинациях турниров Classic

In [498]:
events_classic['event_info'] = events_classic['event_info'].str.replace(' пар', '', regex=True)
events_classic['event_info'] = events_classic['event_info'].str.replace(' / ', ',', regex=True)
events_classic['event_info'] = events_classic['event_info'].str.replace(' уч.', '', regex=True)
events_classic['event_info'] = events_classic['event_info'].str.replace('ы', '', regex=True)
events_classic['event_info'] = events_classic['event_info'].str.replace('а', '', regex=True)
events_classic['event_info'] = events_classic['event_info'].str.replace('nan', '', regex=True)
events_classic['event_info'] = events_classic['event_info'].str.replace('"', '', regex=True)
events_classic['event_info'] = events_classic['event_info'].str.replace('ДнД ', '', regex=True)
events_classic['event_info'] = events_classic['event_info'].str.replace('снят рейт.', 'рейтинг_снят', regex=True)
events_classic['event_info'] = events_classic['event_info'].str.rstrip(',')
events_classic['event_info'] = events_classic['event_info'].str.lstrip(',')

events_classic['event_info'] = events_classic['event_info'].str.replace(',,', ',')
events_classic['event_info'] = events_classic['event_info'].str.replace(',,,', ',')
events_classic['event_info'] = events_classic['event_info'].str.replace(',,,,', ',')
events_classic['event_info'] = events_classic['event_info'].str.replace(',,,,,', ',')
events_classic['event_info'] = events_classic['event_info'].str.replace(',,,,,,', ',')
events_classic['event_info'] = events_classic['event_info'].str.replace(',,', ',')

events_classic[[2, 3, 4, 5, 6, 7, 8]] = events_classic['event_info'].str.split(',', expand=True)

events_classic[['class_1', 'count_1']] = events_classic[2].str.split(' ', 1, expand=True)
events_classic[['class_2', 'count_2']] = events_classic[3].str.split(' ', 1, expand=True)
events_classic[['class_3', 'count_3']] = events_classic[4].str.split(' ', 1, expand=True)
events_classic[['class_4', 'count_4']] = events_classic[5].str.split(' ', 1, expand=True)
events_classic[['class_5', 'count_5']] = events_classic[6].str.split(' ', 1, expand=True)
events_classic[['class_6', 'count_6']] = events_classic[7].str.split(' ', 1, expand=True)
events_classic[['class_7', 'count_7']] = events_classic[8].str.split(' ', 1, expand=True)
events_classic.drop(events_classic.columns[[3, 4, 5, 6, 7, 8, 9]], axis=1, inplace=True)

events_classic.loc[events_classic['class_1'] == 'A', 'class_1'] = 'A'
events_classic.loc[events_classic['class_1'] == 'В', 'class_1'] = 'B'
events_classic.loc[events_classic['class_1'] == 'С', 'class_1'] = 'C'
events_classic.loc[events_classic['class_1'] == 'Е', 'class_1'] = 'E'
events_classic.loc[events_classic['class_2'] == 'A', 'class_2'] = 'A'
events_classic.loc[events_classic['class_2'] == 'В', 'class_2'] = 'B'
events_classic.loc[events_classic['class_2'] == 'С', 'class_2'] = 'C'
events_classic.loc[events_classic['class_2'] == 'Е', 'class_2'] = 'E'
events_classic.loc[events_classic['class_3'] == 'A', 'class_3'] = 'A'
events_classic.loc[events_classic['class_3'] == 'В', 'class_3'] = 'B'
events_classic.loc[events_classic['class_3'] == 'С', 'class_3'] = 'C'
events_classic.loc[events_classic['class_3'] == 'Е', 'class_3'] = 'E'

events_classic.loc[events_classic['class_5'] == 'СBA', 'class_5'] = 'CBA'
events_classic.loc[events_classic['class_4'] == 'В', 'class_4'] = 'B'
events_classic.loc[events_classic['class_4'] == 'BА', 'class_4'] = 'BA'
events_classic.loc[events_classic['class_2'] == 'С', 'class_2'] = 'C'
events_classic.loc[events_classic['class_1'] == 'С', 'class_1'] = 'C'
events_classic.loc[events_classic['class_3'] == 'BC', 'class_3'] = 'CB'
events_classic.loc[events_classic['class_3'] == 'В', 'class_3'] = 'B'
events_classic.loc[events_classic['class_7'] == 'ВA', 'class_7'] = 'BA'

events_classic.loc[events_classic['class_6'] == 'СВА', 'class_6'] = 'CBA'

events_classic.loc[events_classic['class_4'] == 'ВА', 'class_4'] = 'BA'
events_classic.loc[events_classic['class_4'] == 'A+B', 'class_4'] = 'AB'
events_classic.loc[events_classic['class_4'] == 'B+A', 'class_4'] = 'AB'

events_classic.loc[events_classic['class_3'] == 'A+B', 'class_3'] = 'AB'
events_classic.loc[events_classic['class_3'] == 'BAC', 'class_3'] = 'CBA'
events_classic.loc[events_classic['class_3'] == 'D+C+B', 'class_3'] = 'DCB'
events_classic.loc[events_classic['class_3'] == 'BC', 'class_3'] = 'BC'
events_classic.loc[events_classic['class_3'] == 'EC', 'class_3'] = 'EDC'

events_classic['class_7'] = events_classic['class_7'].str.replace('Е-А', 'EDCBA', regex=True)

events_classic['class_6'] = events_classic['class_6'].str.replace('С-А', 'CBA', regex=True)
events_classic['class_6'] = events_classic['class_6'].str.replace('Е-А', 'EDCBA', regex=True)
events_classic['class_6'] = events_classic['class_6'].str.replace('С-В', 'CB', regex=True)
events_classic['class_6'] = events_classic['class_6'].str.replace('Д-А', 'DCBA', regex=True)
events_classic['class_6'] = events_classic['class_6'].str.replace('Д-С', 'DC', regex=True)

events_classic['class_5'] = events_classic['class_5'].str.replace('ДСБ0', 'DCB0', regex=True)
events_classic['class_5'] = events_classic['class_5'].str.replace('ЕДС', 'EDC', regex=True)
events_classic['class_5'] = events_classic['class_5'].str.replace('Д-С', 'DC', regex=True)
events_classic['class_5'] = events_classic['class_5'].str.replace('Е-Д', 'ED', regex=True)
events_classic['class_5'] = events_classic['class_5'].str.replace('С-В', 'CB', regex=True)
events_classic['class_5'] = events_classic['class_5'].str.replace('С-А', 'CBA', regex=True)
events_classic['class_5'] = events_classic['class_5'].str.replace('В-А', 'BA', regex=True)

events_classic['class_4'] = events_classic['class_4'].str.replace('A+B', 'BA', regex=True)
events_classic['class_4'] = events_classic['class_4'].str.replace('B+A', 'BA', regex=True)
events_classic['class_4'] = events_classic['class_4'].str.replace('ЕДСБА', 'EDCBA', regex=True)
events_classic['class_4'] = events_classic['class_4'].str.replace('Е-В', 'EDCB', regex=True)
events_classic['class_4'] = events_classic['class_4'].str.replace('Е-Д', 'ED', regex=True)
events_classic['class_4'] = events_classic['class_4'].str.replace('ДСВ', 'DCB', regex=True)
events_classic['class_4'] = events_classic['class_4'].str.replace('Д-А', 'DCBA', regex=True)
events_classic['class_4'] = events_classic['class_4'].str.replace('C-В', 'CB', regex=True)
events_classic['class_4'] = events_classic['class_4'].str.replace('АБС', 'Absolut', regex=True)
events_classic['class_4'] = events_classic['class_4'].str.replace('B-А', 'BA', regex=True)
events_classic['class_4'] = events_classic['class_4'].str.replace('С-В', 'CB', regex=True)
events_classic['class_4'] = events_classic['class_4'].str.replace('В-В', 'B', regex=True)
events_classic['class_4'] = events_classic['class_4'].str.replace('В-С', 'CB', regex=True)
events_classic['class_4'] = events_classic['class_4'].str.replace('АВС', 'Absolut', regex=True)
events_classic['class_4'] = events_classic['class_4'].str.replace('А-В', 'BA', regex=True)
events_classic['class_4'] = events_classic['class_4'].str.replace('В-А', 'BA', regex=True)
events_classic['class_4'] = events_classic['class_4'].str.replace('С-А', 'CBA', regex=True)
events_classic['class_4'] = events_classic['class_4'].str.replace('Е-А', 'EDCBA', regex=True)

events_classic['class_3'] = events_classic['class_3'].str.replace('C-А', 'CBA', regex=True)
events_classic['class_3'] = events_classic['class_3'].str.replace('Е-Д', 'ED', regex=True)
events_classic['class_3'] = events_classic['class_3'].str.replace('Е-C', 'EDC', regex=True)
events_classic['class_3'] = events_classic['class_3'].str.replace('Абсолют', 'Absolut', regex=True)
events_classic['class_3'] = events_classic['class_3'].str.replace('CBAA', 'CBA', regex=True)
events_classic['class_3'] = events_classic['class_3'].str.replace('D+C+B', 'DCB', regex=True)
events_classic['class_3'] = events_classic['class_3'].str.replace('A+B', 'BA', regex=True)
events_classic['class_3'] = events_classic['class_3'].str.replace('ABC', 'CBA', regex=True)
events_classic['class_3'] = events_classic['class_3'].str.replace('С-А', 'CBA', regex=True)
events_classic['class_3'] = events_classic['class_3'].str.replace('Д-А', 'DCBA', regex=True)
events_classic['class_3'] = events_classic['class_3'].str.replace('Д-С', 'DC', regex=True)
events_classic['class_3'] = events_classic['class_3'].str.replace('В-А', 'BA', regex=True)
events_classic['class_3'] = events_classic['class_3'].str.replace('Е-С', 'EDC', regex=True)
events_classic['class_3'] = events_classic['class_3'].str.replace('С-В', 'CB', regex=True)
events_classic['class_3'] = events_classic['class_3'].str.replace('D-А', 'DCBA', regex=True)

events_classic['class_2'] = events_classic['class_2'].str.replace('Е-Д', 'ED', regex=True)
events_classic['class_2'] = events_classic['class_2'].str.replace('D+C', 'DC', regex=True)
events_classic['class_2'] = events_classic['class_2'].str.replace('D+C', 'DC', regex=True)
events_classic['class_2'] = events_classic['class_2'].str.replace('C+B', 'CB', regex=True)
events_classic['class_2'] = events_classic['class_2'].str.replace('C+B', 'CB', regex=True)
events_classic['class_2'] = events_classic['class_2'].str.replace('BCD', 'DCB', regex=True)
events_classic['class_2'] = events_classic['class_2'].str.replace('ADCB', 'DCBA', regex=True)
events_classic['class_2'] = events_classic['class_2'].str.replace('АBCD', 'DCBA', regex=True)
events_classic['class_2'] = events_classic['class_2'].str.replace('BC', 'CB', regex=True)
events_classic['class_2'] = events_classic['class_2'].str.replace('CD', 'DC', regex=True)
events_classic['class_2'] = events_classic['class_2'].str.replace('ABC', 'CBA', regex=True)
events_classic['class_2'] = events_classic['class_2'].str.replace('АDCB', 'DCBA', regex=True)
events_classic['class_2'] = events_classic['class_2'].str.replace('ACB', 'CBA', regex=True)
events_classic.loc[events_classic['class_2'] == 'C+B', 'class_2'] = 'CB'
events_classic.loc[events_classic['class_2'] == 'D+C', 'class_2'] = 'DC'

events_classic['class_1'] = events_classic['class_1'].str.replace('E/D', 'ED', regex=True)
events_classic['class_1'] = events_classic['class_1'].str.replace('ABCD', 'DCBA', regex=True)
events_classic['class_1'] = events_classic['class_1'].str.replace('BCD', 'DCB', regex=True)

In [504]:
events_classic = events_classic.rename(columns={0 : 'event_name', 1 : 'event_date'})
events_classic = events_classic.sort_index(ascending=False)
events_classic = events_classic.reset_index()
events_classic = events_classic.sort_index()
events_classic = events_classic.drop(columns = 'index')
events_classic = events_classic.reset_index()
events_classic['index'] = events_classic['index']+1
events_classic = events_classic.rename(columns={'index' : 'event_id'})

In [512]:
nomination_1 = events_classic[['event_id', 'event_name', 'event_date', 'event_info', 'class_1', 'count_1']]
nomination_1 = nomination_1.rename(columns={'class_1' : 'nomination', 'count_1' : 'count_dancer'})
nomination_2 = events_classic[['event_id', 'event_name', 'event_date', 'event_info', 'class_2', 'count_2']]
nomination_2= nomination_2.rename(columns={'class_2' : 'nomination', 'count_2' : 'count_dancer'})
nomination_3 = events_classic[['event_id', 'event_name', 'event_date', 'event_info', 'class_3', 'count_3']]
nomination_3 = nomination_3.rename(columns={'class_3' : 'nomination', 'count_3' : 'count_dancer'})
nomination_4 = events_classic[['event_id', 'event_name', 'event_date', 'event_info', 'class_4', 'count_4']]
nomination_4 = nomination_4.rename(columns={'class_4' : 'nomination', 'count_4' : 'count_dancer'})
nomination_5 = events_classic[['event_id', 'event_name', 'event_date', 'event_info', 'class_5', 'count_5']]
nomination_5 = nomination_5.rename(columns={'class_5' : 'nomination', 'count_5' : 'count_dancer'})
nomination_6 = events_classic[['event_id', 'event_name', 'event_date', 'event_info', 'class_6', 'count_6']]
nomination_6 = nomination_6.rename(columns={'class_6' : 'nomination', 'count_6' : 'count_dancer'})
nomination_7 = events_classic[['event_id', 'event_name', 'event_date', 'event_info', 'class_7', 'count_7']]
nomination_7 = nomination_7.rename(columns={'class_7' : 'nomination', 'count_7' : 'count_dancer'})

In [513]:
events_classic = pd.concat([nomination_1, nomination_2, nomination_3, nomination_4, nomination_5, nomination_6, nomination_7], axis=0)
events_classic = events_classic.sort_values(by='event_id').fillna(0)
events_classic = events_classic.loc[events_classic['nomination'] != 0]
events_classic['count_dancer'] = events_classic['count_dancer'].str.lstrip(' ')
events_classic = events_classic.reset_index(drop=True)
events_classic.loc[events_classic['nomination'] == 'BAC', 'nomination'] = 'CBA'
events_classic.loc[events_classic['nomination'] == 'ДСБ0', 'nomination'] = 'DCB0'
events_classic.loc[events_classic['nomination'] == 'BAC', 'nomination'] = 'CBA'
events_classic.loc[events_classic['nomination'] == 'ВА', 'nomination'] = 'BA'
events_classic.loc[events_classic['nomination'] == 'СВ0', 'nomination'] = 'CB0'
events_classic.loc[events_classic['nomination'] == 'СВА', 'nomination'] = 'CBA'

**Комментарий**:
Формирование временной таблицы для дальнейшего получения 'id' конкурсов

In [519]:
events_classic_tmp = events_classic[['event_id', 'event_name', 'event_date']]
events_classic_tmp = events_classic_tmp.drop_duplicates().reset_index(drop=True)
events_classic_tmp['search'] = events_classic_tmp['event_name'] + ' ' + events_classic_tmp['event_date'].astype('str')

**Комментарий**:
Формируем и стандартизируем сведения о названии турниров

In [522]:
events_classic['event_name'] = events_classic['event_name'].str.replace('\n', ' ', regex=True)
events_classic['event_name'] = events_classic['event_name'].str.lstrip(':')
events_classic[['event_name_', 'event_city']] = events_classic['event_name'].str.split(' г.', 1, expand=True)
events_classic['event_name_'] = events_classic['event_name_'].str.lstrip(' ')
events_classic['event_name_'] = events_classic['event_name_'].str.rstrip(' ')
events_classic['event_name_'] = events_classic['event_name_'].str.replace('  ', ' ', regex=True)

In [526]:
events_classic.loc[events_classic['event_name_'] == '1й рейтинговый турнир ХЦ', 'event_name_'] = 'Первый рейтинговый турнир Хастл-Центра'
events_classic.loc[events_classic['event_name_'] == '2-й Cибирский межрегион. конкурс', 'event_name_'] = 'Второй Cибирский межрегиональный конкурс'
events_classic.loc[events_classic['event_name_'] == 'Amber fest', 'event_name_'] = 'Amber Fest'
events_classic.loc[events_classic['event_name_'] == 'Амбер фест 2022', 'event_name_'] = 'Amber Fest'
events_classic.loc[events_classic['event_name_'] == 'BRING THE ACTION 3', 'event_name_'] = 'Bring The Action'
events_classic.loc[events_classic['event_name_'] == 'Bring the action 2', 'event_name_'] = 'Bring The Action'
events_classic.loc[events_classic['event_name_'] == 'Bring the action 1', 'event_name_'] = 'Bring The Action'
events_classic.loc[events_classic['event_name_'] == 'CRYSTAL DANCE', 'event_name_'] = 'Crystal Dance'
events_classic.loc[events_classic['event_name_'] == 'DANCEVISION 2022', 'event_name_'] = 'DANCEVISION'
events_classic.loc[events_classic['event_name_'] == 'Dance Revolution Cup', 'event_name_'] = 'Dance Revolution Cup'
events_classic.loc[events_classic['event_name_'] == 'Dance Revolution Cup 2018', 'event_name_'] = 'Dance Revolution Cup'
events_classic.loc[events_classic['event_name_'] == 'Dance weekend', 'event_name_'] = 'Танцевальный уикенд'
events_classic.loc[events_classic['event_name_'] == 'DanceWeekend', 'event_name_'] = 'Танцевальный уикенд'
events_classic.loc[events_classic['event_name_'] == 'Танцевальный Weekend', 'event_name_'] = 'Танцевальный уикенд'
events_classic.loc[events_classic['event_name_'] == 'Танцевальный Weekend 2021', 'event_name_'] = 'Танцевальный уикенд'
events_classic.loc[events_classic['event_name_'] == 'Танцевальный weekend', 'event_name_'] = 'Танцевальный уикенд'
events_classic.loc[events_classic['event_name_'] == 'Танцевальный weekend 2020 СПб', 'event_name_'] = 'Танцевальный уикенд'
events_classic.loc[events_classic['event_name_'] == 'Танцевальный уикенд', 'event_name_'] = 'Танцевальный уикенд'
events_classic.loc[events_classic['event_name_'] == 'Танц. Weekend', 'event_name_'] = 'Танцевальный уикенд'
events_classic.loc[events_classic['event_name_'] == 'EuroDance - 3', 'event_name_'] = 'EuroDance'
events_classic.loc[events_classic['event_name_'] == 'EuroDance 2', 'event_name_'] = 'EuroDance'
events_classic.loc[events_classic['event_name_'] == 'For Friends Only 2013', 'event_name_'] = 'For Friends Only'
events_classic.loc[events_classic['event_name_'] == 'H&D Festival Cup', 'event_name_'] = 'H&D Cup'
events_classic.loc[events_classic['event_name_'] == 'HD Cup', 'event_name_'] = 'H&D Cup'
events_classic.loc[events_classic['event_name_'] == 'HD Cup 2022', 'event_name_'] = 'H&D Cup'
events_classic.loc[events_classic['event_name_'] == 'Hustle&Discofox Festival Cup', 'event_name_'] = 'H&D Cup'
events_classic.loc[events_classic['event_name_'] == 'HUSTLE DanсeFest', 'event_name_'] = 'Hustle DanсeFest'
events_classic.loc[events_classic['event_name_'] == 'HOT WINTER CUP', 'event_name_'] = 'Hot Winter Cup'
events_classic.loc[events_classic['event_name_'] == 'Hot Winter', 'event_name_'] = 'Hot Winter Cup'
events_classic.loc[events_classic['event_name_'] == 'Halloween Cup 2016', 'event_name_'] = 'Halloween Cup'
events_classic.loc[events_classic['event_name_'] == 'ICE CUP 2018', 'event_name_'] = 'Ice Cup'
events_classic.loc[events_classic['event_name_'] == 'IV КУБОК МИНСКА 2012', 'event_name_'] = 'Кубок Минска'
events_classic.loc[events_classic['event_name_'] == 'VI Открытый Кубок', 'event_name_'] = 'Кубок Минска'
events_classic.loc[events_classic['event_name_'] == 'Leningrad Cup 2016', 'event_name_'] = 'Leningrad Cup'
events_classic.loc[events_classic['event_name_'] == 'Leningrad cup', 'event_name_'] = 'Leningrad Cup'
events_classic.loc[events_classic['event_name_'] == 'Music-UP Competition', 'event_name_'] = 'MusicUP Competition'
events_classic.loc[events_classic['event_name_'] == 'MusicUP competition', 'event_name_'] = 'MusicUP Competition'
events_classic.loc[events_classic['event_name_'] == 'MusicUp', 'event_name_'] = 'MusicUP Competition'
events_classic.loc[events_classic['event_name_'] == 'Nord Cup 2016', 'event_name_'] = 'Nord Cup'
events_classic.loc[events_classic['event_name_'] == 'NordCup 2018', 'event_name_'] = 'Nord Cup'
events_classic.loc[events_classic['event_name_'] == 'NordCup 2019', 'event_name_'] = 'Nord Cup'
events_classic.loc[events_classic['event_name_'] == 'NordCup 2021', 'event_name_'] = 'Nord Cup'
events_classic.loc[events_classic['event_name_'] == 'NordCup 2022', 'event_name_'] = 'Nord Cup'
events_classic.loc[events_classic['event_name_'] == 'OCTOBER Fest', 'event_name_'] = 'October Fest'
events_classic.loc[events_classic['event_name_'] == 'Pacific Motion 2018', 'event_name_'] = 'Pacific Motion Cup'
events_classic.loc[events_classic['event_name_'] == 'Pacific Motion 2019', 'event_name_'] = 'Pacific Motion Cup'
events_classic.loc[events_classic['event_name_'] == 'Pacific motion Cup 2022', 'event_name_'] = 'Pacific Motion Cup'
events_classic.loc[events_classic['event_name_'] == 'REDcup,', 'event_name_'] = 'REDcup'
events_classic.loc[events_classic['event_name_'] == "Rock'N'Dance CUP 2019", 'event_name_'] = "Rock'n'Dance Cup"
events_classic.loc[events_classic['event_name_'] == "Rock'n'Dance CUP 2017", 'event_name_'] = "Rock'n'Dance Cup"
events_classic.loc[events_classic['event_name_'] == "Rock'n'Dance CUP", 'event_name_'] = "Rock'n'Dance Cup"
events_classic.loc[events_classic['event_name_'] == "Rock'n'Dance CUP 2018", 'event_name_'] = "Rock'n'Dance Cup"
events_classic.loc[events_classic['event_name_'] == "Rock'n'Dance CUP 2020", 'event_name_'] = "Rock'n'Dance Cup"
events_classic.loc[events_classic['event_name_'] == "Rock'n'Dance CUP 2021", 'event_name_'] = "Rock'n'Dance Cup"
events_classic.loc[events_classic['event_name_'] == 'SNEG-i-RHYTHM', 'event_name_'] = 'Sneg-i-Rhythm'
events_classic.loc[events_classic['event_name_'] == 'SIBERIAN JAM', 'event_name_'] = 'Siberian Jam'
events_classic.loc[events_classic['event_name_'] == 'Siberian jam', 'event_name_'] = 'Siberian Jam'
events_classic.loc[events_classic['event_name_'] == 'So You think you can dance', 'event_name_'] = 'So you think you can dance'
events_classic.loc[events_classic['event_name_'] == 'So you think you can Dance', 'event_name_'] = 'So you think you can dance'
events_classic.loc[events_classic['event_name_'] == 'Starship 2017', 'event_name_'] = 'StarShip'
events_classic.loc[events_classic['event_name_'] == 'Старшип 2016', 'event_name_'] = 'StarShip'
events_classic.loc[events_classic['event_name_'] == 'Step by Step Красноярск', 'event_name_'] = 'Step by Step'
events_classic.loc[events_classic['event_name_'] == 'Summer Dance Cup 2021', 'event_name_'] = 'Summer Dance Cup'
events_classic.loc[events_classic['event_name_'] == 'Summer Dance Cup 2019', 'event_name_'] = 'Summer Dance Cup'
events_classic.loc[events_classic['event_name_'] == 'Summer Dance Cup 2018', 'event_name_'] = 'Summer Dance Cup'
events_classic.loc[events_classic['event_name_'] == 'SunnyFest 2021', 'event_name_'] = 'SunnyFest'
events_classic.loc[events_classic['event_name_'] == 'SunnyFest 2022', 'event_name_'] = 'SunnyFest'
events_classic.loc[events_classic['event_name_'] == 'Upgrade For You 2022', 'event_name_'] = 'Upgrade for you'
events_classic.loc[events_classic['event_name_'] == 'UralFest 2016', 'event_name_'] = 'UralFest'
events_classic.loc[events_classic['event_name_'] == 'UralFest 2021', 'event_name_'] = 'UralFest'
events_classic.loc[events_classic['event_name_'] == 'XmasCup 2020', 'event_name_'] = 'XmasCup'
events_classic.loc[events_classic['event_name_'] == 'XmasCup 2018', 'event_name_'] = 'XmasCup'
events_classic.loc[events_classic['event_name_'] == 'XmasCup 2019', 'event_name_'] = 'XmasCup'
events_classic.loc[events_classic['event_name_'] == 'XmasCup 2021', 'event_name_'] = 'XmasCup'
events_classic.loc[events_classic['event_name_'] == 'YES!Volga Dance Сamp', 'event_name_'] = 'YES! Volga Dance Camp'
events_classic.loc[events_classic['event_name_'] == 'YES! Volga Dance Сamp', 'event_name_'] = 'YES! Volga Dance Camp'
events_classic.loc[events_classic['event_name_'] == 'Буревестник 2011', 'event_name_'] = 'Кубок Буревестника'
events_classic.loc[events_classic['event_name_'] == 'Кубок Буревестника 2013', 'event_name_'] = 'Кубок Буревестника'
events_classic.loc[events_classic['event_name_'] == 'Весенние Огни', 'event_name_'] = 'Весенние огни'
events_classic.loc[events_classic['event_name_'] == 'Весенние огни 2018', 'event_name_'] = 'Весенние огни'
events_classic.loc[events_classic['event_name_'] == 'Весенние огни 2021', 'event_name_'] = 'Весенние огни'
events_classic.loc[events_classic['event_name_'] == 'Весенние огни 2022', 'event_name_'] = 'Весенние огни'
events_classic.loc[events_classic['event_name_'] == 'Весенний кубок Движения 2019', 'event_name_'] = 'Весенний Кубок Движения'
events_classic.loc[events_classic['event_name_'] == 'Весенний кубок Движения', 'event_name_'] = 'Весенний Кубок Движения'
events_classic.loc[events_classic['event_name_'] == 'Весенний кубок 2009', 'event_name_'] = 'Весенний Кубок Vestadance'
events_classic.loc[events_classic['event_name_'] == 'Весенний Кубок', 'event_name_'] = 'Весенний Кубок Vestadance'
events_classic.loc[events_classic['event_name_'] == 'Весенний Кубок VestaDance', 'event_name_'] = 'Весенний Кубок Vestadance'
events_classic.loc[events_classic['event_name_'] == 'Grand Prix', 'event_name_'] = 'Grand Prix: Masters Cup'
events_classic.loc[events_classic['event_name_'] == 'Premier Cup', 'event_name_'] = 'Grand Prix: Premier Cup'
events_classic.loc[events_classic['event_name_'] == 'Star Cup', 'event_name_'] = 'Grand Prix: Star Cup'
events_classic.loc[events_classic['event_name_'] == 'Дебют 2009', 'event_name_'] = 'Дебют'
events_classic.loc[events_classic['event_name_'] == 'Дебют 3', 'event_name_'] = 'Дебют'
events_classic.loc[events_classic['event_name_'] == 'Дебют-2011', 'event_name_'] = 'Дебют'
events_classic.loc[events_classic['event_name_'] == 'Кубок VestaDance "Дебют-2"', 'event_name_'] = 'Дебют'
events_classic.loc[events_classic['event_name_'] == 'Держи Ритм 15', 'event_name_'] = 'Держи Ритм'
events_classic.loc[events_classic['event_name_'] == 'Держи Ритм 11', 'event_name_'] = 'Держи Ритм'
events_classic.loc[events_classic['event_name_'] == 'Держи Ритм 12', 'event_name_'] = 'Держи Ритм'
events_classic.loc[events_classic['event_name_'] == 'Держи Ритм 13', 'event_name_'] = 'Держи Ритм'
events_classic.loc[events_classic['event_name_'] == 'Держи Ритм 14', 'event_name_'] = 'Держи Ритм'
events_classic.loc[events_classic['event_name_'] == 'Держи Ритм 16', 'event_name_'] = 'Держи Ритм'
events_classic.loc[events_classic['event_name_'] == 'Держи Ритм 2', 'event_name_'] = 'Держи Ритм'
events_classic.loc[events_classic['event_name_'] == 'Держи Ритм 9', 'event_name_'] = 'Держи Ритм'
events_classic.loc[events_classic['event_name_'] == 'Держи ритм', 'event_name_'] = 'Держи Ритм'
events_classic.loc[events_classic['event_name_'] == 'Динамиада 2015', 'event_name_'] = 'Динамиада'
events_classic.loc[events_classic['event_name_'] == 'Динамиада 2019', 'event_name_'] = 'Динамиада'
events_classic.loc[events_classic['event_name_'] == 'Динамиада 2021', 'event_name_'] = 'Динамиада'
events_classic.loc[events_classic['event_name_'] == 'Динамиада 2022', 'event_name_'] = 'Динамиада'
events_classic.loc[events_classic['event_name_'] == 'Звезды Хастла 2009', 'event_name_'] = 'Звезды Хастлa'
events_classic.loc[events_classic['event_name_'] == 'Звезды Хастла 2008', 'event_name_'] = 'Звезды Хастлa'
events_classic.loc[events_classic['event_name_'] == 'Звезды Хастла', 'event_name_'] = 'Звезды Хастлa'
events_classic.loc[events_classic['event_name_'] == 'Звезды Хастл', 'event_name_'] = 'Звезды Хастлa'
events_classic.loc[events_classic['event_name_'] == 'Зимн.Кубок СПБ', 'event_name_'] = 'Зимний Кубок Санкт-Петербурга'
events_classic.loc[events_classic['event_name_'] == 'Зимний Кубок СПб', 'event_name_'] = 'Зимний Кубок Санкт-Петербурга'
events_classic.loc[events_classic['event_name_'] == 'Зимний Кубок Академии Танца', 'event_name_'] = 'Зимний Кубок Академии Танца'
events_classic.loc[events_classic['event_name_'] == 'Зимний кубок Vestadance', 'event_name_'] = 'Зимний Кубок Vestadance'
events_classic.loc[events_classic['event_name_'] == 'Зимний Кубок Сибири 2013', 'event_name_'] = 'Зимний Кубок Сибири'
events_classic.loc[events_classic['event_name_'] == 'Зимний Кубок 2013', 'event_name_'] = 'Зимний Кубок'
events_classic.loc[events_classic['event_name_'] == 'Зимний Кубок 2015', 'event_name_'] = 'Зимний Кубок'
events_classic.loc[events_classic['event_name_'] == 'Зимний кубок', 'event_name_'] = 'Зимний Кубок'
#Зимний Кубок проводился в Минске и Москве, после изменения московскому надо добавить Vestadance
events_classic.loc[events_classic['event_name_'] == 'Золото Осени (СПб)', 'event_name_'] = 'Золото Осени'
events_classic.loc[events_classic['event_name_'] == 'Золото осени 2011', 'event_name_'] = 'Золото Осени'
#Поправить город на Санкт-Петербург
events_classic.loc[events_classic['event_name_'] == 'КУБОК АСХ 2019', 'event_name_'] = 'Кубок АСХ'
events_classic.loc[events_classic['event_name_'] == 'Кубок АСХ 2017', 'event_name_'] = 'Кубок АСХ'
events_classic.loc[events_classic['event_name_'] == 'Кубок АСХ 2012', 'event_name_'] = 'Кубок АСХ'
events_classic.loc[events_classic['event_name_'] == 'Кубок АСХ 2013', 'event_name_'] = 'Кубок АСХ'
events_classic.loc[events_classic['event_name_'] == 'Кубок АСХ 2015', 'event_name_'] = 'Кубок АСХ'
events_classic.loc[events_classic['event_name_'] == 'Кубок АСХ 2016', 'event_name_'] = 'Кубок АСХ'
events_classic.loc[events_classic['event_name_'] == 'Кубок АСХ 2020', 'event_name_'] = 'Кубок АСХ'
events_classic.loc[events_classic['event_name_'] == 'Кубок АСХ 2009', 'event_name_'] = 'Кубок АСХ'
events_classic.loc[events_classic['event_name_'] == 'Кубок АСХ 2018', 'event_name_'] = 'Кубок АСХ'
events_classic.loc[events_classic['event_name_'] == 'Кубок АСХ 2021', 'event_name_'] = 'Кубок АСХ'
events_classic.loc[events_classic['event_name_'] == 'Кубок АСХ 2022', 'event_name_'] = 'Кубок АСХ'
events_classic.loc[events_classic['event_name_'] == 'КУБОК ЛЕТА', 'event_name_'] = 'Кубок Лета'
events_classic.loc[events_classic['event_name_'] == 'КУБОК ЛЕТА 2020', 'event_name_'] = 'Кубок Лета'
events_classic.loc[events_classic['event_name_'] == 'Кубок Лета 2016', 'event_name_'] = 'Кубок Лета'
events_classic.loc[events_classic['event_name_'] == 'Кубок Лета 2012', 'event_name_'] = 'Кубок Лета'
events_classic.loc[events_classic['event_name_'] == 'Кубок Лета 2010 (2й тур)', 'event_name_'] = 'Кубок Лета'
events_classic.loc[events_classic['event_name_'] == 'Кубок Лета 2011 (г.СПб)', 'event_name_'] = 'Кубок Лета'
#Поправить город на Санкт-Петербург
events_classic.loc[events_classic['event_name_'] == 'Кубок Лета 2010', 'event_name_'] = 'Кубок Лета'
events_classic.loc[events_classic['event_name_'] == 'Кубок Лета 2018', 'event_name_'] = 'Кубок Лета'
events_classic.loc[events_classic['event_name_'] == 'Кубок Лета 2019', 'event_name_'] = 'Кубок Лета'
events_classic.loc[events_classic['event_name_'] == 'Кубок Лета, 4 тур (СПб)', 'event_name_'] = 'Кубок Лета'
#до 2014 года турнир с таким названием проходил только в СПб, после 2014 только в Москве
events_classic.loc[events_classic['event_name_'] == 'КУБОК МОРСКОЙ СЛАВЫ', 'event_name_'] = 'Кубок Морской Славы'
events_classic.loc[events_classic['event_name_'] == 'Кубок морской славы', 'event_name_'] = 'Кубок Морской Славы'
events_classic.loc[events_classic['event_name_'] == 'Кубок морской славы 2019', 'event_name_'] = 'Кубок Морской Славы'
events_classic.loc[events_classic['event_name_'] == 'Кубок морской славы', 'event_name_'] = 'Кубок Морской Славы'
events_classic.loc[events_classic['event_name_'] == 'Командное Первенство 2015', 'event_name_'] = 'Командное Первенство'
events_classic.loc[events_classic['event_name_'] == 'Командное Первенство 2020', 'event_name_'] = 'Командное Первенство'
events_classic.loc[events_classic['event_name_'] == 'Командное первенство', 'event_name_'] = 'Командное Первенство'
events_classic.loc[events_classic['event_name_'] == 'Командное первенство 2011', 'event_name_'] = 'Командное Первенство'
events_classic.loc[events_classic['event_name_'] == 'Командное первенство 2012', 'event_name_'] = 'Командное Первенство'
events_classic.loc[events_classic['event_name_'] == 'Командное Первенство 2016', 'event_name_'] = 'Командное Первенство'
events_classic.loc[events_classic['event_name_'] == 'Командное Первенство 2017', 'event_name_'] = 'Командное Первенство'
events_classic.loc[events_classic['event_name_'] == 'Командное Первенство 2018', 'event_name_'] = 'Командное Первенство'
events_classic.loc[events_classic['event_name_'] == 'Командное Первенство 2021', 'event_name_'] = 'Командное Первенство'
events_classic.loc[events_classic['event_name_'] == 'Космос 2011', 'event_name_'] = 'Космос'
events_classic.loc[events_classic['event_name_'] == 'Космос 2012', 'event_name_'] = 'Космос'
events_classic.loc[events_classic['event_name_'] == 'Куб.Великого Новгорода', 'event_name_'] = 'Кубок Великого Новгорода'
events_classic.loc[events_classic['event_name_'] == 'Куб.Ренессанса', 'event_name_'] = 'Кубок Ренессанса'
events_classic.loc[events_classic['event_name_'] == 'Кубок Ренессанса 2011', 'event_name_'] = 'Кубок Ренессанса'
events_classic.loc[events_classic['event_name_'] == 'Кубок Ренессанса 2013', 'event_name_'] = 'Кубок Ренессанса'
events_classic.loc[events_classic['event_name_'] == 'Кубок Ренессанса 2014', 'event_name_'] = 'Кубок Ренессанса'
events_classic.loc[events_classic['event_name_'] == 'Кубок В.Новгорода 2011', 'event_name_'] = 'Кубок Великого Новгорода'
events_classic.loc[events_classic['event_name_'] == 'Кубок Геленджика 2015', 'event_name_'] = 'Кубок Геленджика'
events_classic.loc[events_classic['event_name_'] == 'Кубок Геленджика 2018', 'event_name_'] = 'Кубок Геленджика'
events_classic.loc[events_classic['event_name_'] == 'Кубок Екатеринбурга 2015', 'event_name_'] = 'Кубок Екатеринбурга'
events_classic.loc[events_classic['event_name_'] == 'Кубок Екатеринбурга 2018', 'event_name_'] = 'Кубок Екатеринбурга'
events_classic.loc[events_classic['event_name_'] == 'Кубок Красноярска 2018', 'event_name_'] = 'Кубок Красноярска'
events_classic.loc[events_classic['event_name_'] == 'Кубок КХК "Движение" Красноярск', 'event_name_'] = 'Кубок Движения Красноярск'
events_classic.loc[events_classic['event_name_'] == 'Кубок Огненного Лиса 2016', 'event_name_'] = 'Кубок Огненного Лиса'
events_classic.loc[events_classic['event_name_'] == 'Кубок Огня 2019', 'event_name_'] = 'Кубок Огня'
events_classic.loc[events_classic['event_name_'] == 'Кубок Ростова 2017', 'event_name_'] = 'Кубок Ростова'
events_classic.loc[events_classic['event_name_'] == 'Кубок Ростова-на-Дону 2016', 'event_name_'] = 'Кубок Ростова'
events_classic.loc[events_classic['event_name_'] == 'Кубок по Хастлу', 'event_name_'] = 'Кубок Ростова'
events_classic.loc[events_classic['event_name_'] == 'Кубок Сев.Стол.', 'event_name_'] = 'Кубок Северной Столицы'
events_classic.loc[events_classic['event_name_'] == 'Кубок Северо- Запада', 'event_name_'] = 'Кубок Северо-Запада'
events_classic.loc[events_classic['event_name_'] == 'Кубок Чемпионов 2019', 'event_name_'] = 'Кубок Чемпионов'
events_classic.loc[events_classic['event_name_'] == 'Кубок Чемпионов 2020', 'event_name_'] = 'Кубок Чемпионов'
events_classic.loc[events_classic['event_name_'] == 'Кубок Чемпионов 2', 'event_name_'] = 'Кубок Чемпионов'
events_classic.loc[events_classic['event_name_'] == 'Кубок Чемпионов 2017', 'event_name_'] = 'Кубок Чемпионов'
events_classic.loc[events_classic['event_name_'] == 'Кубок Чемпионов 2018', 'event_name_'] = 'Кубок Чемпионов'
events_classic.loc[events_classic['event_name_'] == 'Кубок Св.Вал.', 'event_name_'] = 'Кубок Святого Валентина'
events_classic.loc[events_classic['event_name_'] == 'Кубок Спартака 2008', 'event_name_'] = 'Кубок Спартака'
events_classic.loc[events_classic['event_name_'] == 'Кубок Школы Чемпионов 2022', 'event_name_'] = 'Кубок Школы Чемпионов'
events_classic.loc[events_classic['event_name_'] == 'Кубок Южной Столицы 2019', 'event_name_'] = 'Кубок Южной Столицы'
events_classic.loc[events_classic['event_name_'] == 'Кубок в Новосиб.', 'event_name_'] = 'Открытый Кубок Новосибирска'
events_classic.loc[events_classic['event_name_'] == 'Летнее первенство 2008', 'event_name_'] = 'Летний Кубок Vestadance'
events_classic.loc[events_classic['event_name_'] == 'Летний кубок Весты', 'event_name_'] = 'Летний Кубок Vestadance'
events_classic.loc[events_classic['event_name_'] == 'Летний кубок Vestadance', 'event_name_'] = 'Летний Кубок Vestadance'
events_classic.loc[events_classic['event_name_'] == 'Летние ночи 2019', 'event_name_'] = 'Летние ночи'
events_classic.loc[events_classic['event_name_'] == 'Летние ночи 2021', 'event_name_'] = 'Летние ночи'
events_classic.loc[events_classic['event_name_'] == 'Летние ночи 2022', 'event_name_'] = 'Летние ночи'
events_classic.loc[events_classic['event_name_'] == 'Летний Кубок 2016', 'event_name_'] = 'Летний Кубок'
events_classic.loc[events_classic['event_name_'] == 'Летний Кубок Минска', 'event_name_'] = 'Летний Кубок'
events_classic.loc[events_classic['event_name_'] == 'Летний кубок', 'event_name_'] = 'Летний Кубок'
events_classic.loc[events_classic['event_name_'] == 'Летний Кубок Новосибирска 2018', 'event_name_'] = 'Летний Кубок Новосибирска'
events_classic.loc[events_classic['event_name_'] == 'Летний кубок Академии Танца', 'event_name_'] = 'Летний Кубок Академии Танца'
events_classic.loc[events_classic['event_name_'] == 'Летний кубок (г.Саратов)', 'event_name_'] = 'Летний Кубок Академии Танца'
#Проверить город, должен быть Саратов
events_classic.loc[events_classic['event_name_'] == 'МОСКОВСКИЙ БИТ 2021', 'event_name_'] = 'Московский бит'
events_classic.loc[events_classic['event_name_'] == 'Майский Бал N Club’а 2008', 'event_name_'] = "Майский Бал N'Club"
events_classic.loc[events_classic['event_name_'] == 'На пути к звездам 2018', 'event_name_'] = 'На пути к звездам'
events_classic.loc[events_classic['event_name_'] == 'На пути к Звездам', 'event_name_'] = 'На пути к звездам'
events_classic.loc[events_classic['event_name_'] == 'На пути к звездам Красноярск', 'event_name_'] = 'На пути к звездам'
events_classic.loc[events_classic['event_name_'] == 'На пути к звёздам', 'event_name_'] = 'На пути к звездам'
events_classic.loc[events_classic['event_name_'] == 'Надежды России 2015', 'event_name_'] = 'Надежды России'
events_classic.loc[events_classic['event_name_'] == 'Надежды России 2019', 'event_name_'] = 'Надежды России'
events_classic.loc[events_classic['event_name_'] == 'Надежды России 2020', 'event_name_'] = 'Надежды России'
events_classic.loc[events_classic['event_name_'] == 'Надежды России 2021', 'event_name_'] = 'Надежды России'
events_classic.loc[events_classic['event_name_'] == 'Надежды России 2016', 'event_name_'] = 'Надежды России'
events_classic.loc[events_classic['event_name_'] == 'Надежды России 2017', 'event_name_'] = 'Надежды России'
events_classic.loc[events_classic['event_name_'] == 'Надежды России 2018', 'event_name_'] = 'Надежды России'
events_classic.loc[events_classic['event_name_'] == 'Новая волна 2103', 'event_name_'] = 'Новая волна'
events_classic.loc[events_classic['event_name_'] == 'Новая волна 2012', 'event_name_'] = 'Новая волна'
events_classic.loc[events_classic['event_name_'] == 'Огни Большого Города 2021', 'event_name_'] = 'Огни Большого Города'
events_classic.loc[events_classic['event_name_'] == 'Огни большого', 'event_name_'] = 'Огни Большого Города'
#проверить город в Огнях большого города
events_classic.loc[events_classic['event_name_'] == 'Огни Ростова 2017', 'event_name_'] = 'Огни Ростова'
events_classic.loc[events_classic['event_name_'] == 'Огни Ростова 2018', 'event_name_'] = 'Огни Ростова'
events_classic.loc[events_classic['event_name_'] == 'Осенний кубок Ренессанса', 'event_name_'] = 'Осенний Кубок Ренессанса'
events_classic.loc[events_classic['event_name_'] == 'Осенний кубок Движения', 'event_name_'] = 'Осенний Кубок Движения'
events_classic.loc[events_classic['event_name_'] == 'Осенний кубок Движения 2.0', 'event_name_'] = 'Осенний Кубок Движения'
events_classic.loc[events_classic['event_name_'] == 'Осенний кубок Vestadance', 'event_name_'] = 'Осенний Кубок Vestadance'
events_classic.loc[events_classic['event_name_'] == 'Осенний кубок 2012', 'event_name_'] = 'Осенний Кубок'
events_classic.loc[events_classic['event_name_'] == 'Осенний кубок Априори', 'event_name_'] = 'Осенний Кубок Априори'
events_classic.loc[events_classic['event_name_'] == 'Откр.Кубок Новосибирска', 'event_name_'] = 'Открытый Кубок Новосибирска'
events_classic.loc[events_classic['event_name_'] == 'Открытие сезона', 'event_name_'] = 'Открытие Сезона'
events_classic.loc[events_classic['event_name_'] == 'Открытие сезона 2016-2017', 'event_name_'] = 'Открытие Сезона'
events_classic.loc[events_classic['event_name_'] == 'Открытие Года 2012', 'event_name_'] = 'Открытие Года'
events_classic.loc[events_classic['event_name_'] == 'Открытый Кубок 2015', 'event_name_'] = 'Открытый Кубок'
events_classic.loc[events_classic['event_name_'] == 'Открытый Кубок Движения', 'event_name_'] = 'Открытый Кубок Движения'
events_classic.loc[events_classic['event_name_'] == 'Открытый Кубок 2015', 'event_name_'] = 'Открытый Кубок'
events_classic.loc[events_classic['event_name_'] == 'Открытый Кубок 2016', 'event_name_'] = 'Открытый Кубок'
events_classic.loc[events_classic['event_name_'] == 'Открытый кубок Движения', 'event_name_'] = 'Открытый Кубок Движения'
events_classic.loc[events_classic['event_name_'] == 'Открытый кубок Новосибирска', 'event_name_'] = 'Открытый Кубок Новосибирска'
events_classic.loc[events_classic['event_name_'] == 'Открытый межрегион. Кубок Сибири', 'event_name_'] = 'Открытый межрегиональный Кубок Сибири'
events_classic.loc[events_classic['event_name_'] == 'Открытый кубок ЕД (Красноярск)', 'event_name_'] = 'Открытый Кубок для ED классов'
events_classic.loc[events_classic['event_name_'] == 'Открытый кубок для E+D', 'event_name_'] = 'Открытый Кубок для ED классов'
events_classic.loc[events_classic['event_name_'] == 'Открытый турнир', 'event_name_'] = 'Открытый турнир для ED классов'
events_classic.loc[events_classic['event_name_'] == 'Открытый турнир E+D', 'event_name_'] = 'Открытый турнир для ED классов'
events_classic.loc[events_classic['event_name_'] == 'Открытый турнир E+D класс', 'event_name_'] = 'Открытый турнир для ED классов'
events_classic.loc[events_classic['event_name_'] == 'Открытый турнир E+D классов', 'event_name_'] = 'Открытый турнир для ED классов'
events_classic.loc[events_classic['event_name_'] == 'Открытый турнир E-D классов', 'event_name_'] = 'Открытый турнир для ED классов'
events_classic.loc[events_classic['event_name_'] == 'Открытый турнир ED', 'event_name_'] = 'Открытый турнир для ED классов'
events_classic.loc[events_classic['event_name_'] == 'Открытый турнир Е+D', 'event_name_'] = 'Открытый турнир для ED классов'
events_classic.loc[events_classic['event_name_'] == 'Открытый турнир Е+D 2022', 'event_name_'] = 'Открытый турнир для ED классов'
events_classic.loc[events_classic['event_name_'] == 'Открытый турнир Е+D классов', 'event_name_'] = 'Открытый турнир для ED классов'
events_classic.loc[events_classic['event_name_'] == 'Открытый турнир для E+D', 'event_name_'] = 'Открытый турнир для ED классов'
events_classic.loc[events_classic['event_name_'] == 'Открытый турнир для Е+D', 'event_name_'] = 'Открытый турнир для ED классов'
events_classic.loc[events_classic['event_name_'] == 'Открытый турнир для Е+D классов', 'event_name_'] = 'Открытый турнир для ED классов'
events_classic.loc[events_classic['event_name_'] == 'Открытый турнир Красноярск', 'event_name_'] = 'Открытый турнир для ED классов'
events_classic.loc[events_classic['event_name_'] == 'Турнир Е+Д классов', 'event_name_'] = 'Турнир для ED классов'
events_classic.loc[events_classic['event_name_'] == 'Турнир для E+D класса.', 'event_name_'] = 'Турнир для ED классов'
events_classic.loc[events_classic['event_name_'] == 'ПЕРЕСТРОЙКА 2020', 'event_name_'] = 'Перестройка'
events_classic.loc[events_classic['event_name_'] == 'Перезагрузка 2015', 'event_name_'] = 'Перезагрузка'
events_classic.loc[events_classic['event_name_'] == 'Перезагрузка 2016', 'event_name_'] = 'Перезагрузка'
events_classic.loc[events_classic['event_name_'] == 'Перезагрузка 2022', 'event_name_'] = 'Перезагрузка'
events_classic.loc[events_classic['event_name_'] == 'Позитивный Кубок (СПб)', 'event_name_'] = 'Позитивный Кубок'
#проверить город, СПб
events_classic.loc[events_classic['event_name_'] == 'Полет 2009', 'event_name_'] = 'Полет'
events_classic.loc[events_classic['event_name_'] == 'Полёт-2', 'event_name_'] = 'Полет'
events_classic.loc[events_classic['event_name_'] == 'Простые движ. (СПб)', 'event_name_'] = 'Простые движения'
#проверить город, СПб
events_classic.loc[events_classic['event_name_'] == 'Самоцветы Урала 2018,', 'event_name_'] = 'Самоцветы Урала'
events_classic.loc[events_classic['event_name_'] == 'Самоцветы Урала 2020', 'event_name_'] = 'Самоцветы Урала'
events_classic.loc[events_classic['event_name_'] == 'Самоцветы Урала 2021,', 'event_name_'] = 'Самоцветы Урала'
events_classic.loc[events_classic['event_name_'] == 'СуперЛига Сибири 2017', 'event_name_'] = 'СуперЛига Сибири'
events_classic.loc[events_classic['event_name_'] == 'СуперЛига Сибири 2018', 'event_name_'] = 'СуперЛига Сибири'
events_classic.loc[events_classic['event_name_'] == 'СуперЛига Сибири 2019', 'event_name_'] = 'СуперЛига Сибири'
events_classic.loc[events_classic['event_name_'] == 'СуперЛига Сибири 2021', 'event_name_'] = 'СуперЛига Сибири'
events_classic.loc[events_classic['event_name_'] == 'СуперЛига Сибири 2022', 'event_name_'] = 'СуперЛига Сибири'
events_classic.loc[events_classic['event_name_'] == 'Танцевальная волна 2015', 'event_name_'] = 'Танцевальная волна'
events_classic.loc[events_classic['event_name_'] == 'Танцевальная волна 2016', 'event_name_'] = 'Танцевальная волна'
events_classic.loc[events_classic['event_name_'] == 'Танцевальный Район', 'event_name_'] = 'Танцевальный район'
events_classic.loc[events_classic['event_name_'] == 'Танцующий', 'event_name_'] = 'Танцующий город'
events_classic.loc[events_classic['event_name_'] == 'Ураган 2016', 'event_name_'] = 'Ураган'
events_classic.loc[events_classic['event_name_'] == 'Ураган 2017', 'event_name_'] = 'Ураган'
events_classic.loc[events_classic['event_name_'] == 'Ураган 2018', 'event_name_'] = 'Ураган'
events_classic.loc[events_classic['event_name_'] == 'Ураган 2019', 'event_name_'] = 'Ураган'
events_classic.loc[events_classic['event_name_'] == 'Ураган 2021', 'event_name_'] = 'Ураган'
events_classic.loc[events_classic['event_name_'] == 'Ураган 2022', 'event_name_'] = 'Ураган'
events_classic.loc[events_classic['event_name_'] == 'Февральский Марафон', 'event_name_'] = 'Февральский марафон'
events_classic.loc[events_classic['event_name_'] == 'Февральский марафон 2016', 'event_name_'] = 'Февральский марафон'
events_classic.loc[events_classic['event_name_'] == 'Февральский марафон,', 'event_name_'] = 'Февральский марафон'
events_classic.loc[events_classic['event_name_'] == 'ЧЕМПИОНАТ РОССИИ 2021', 'event_name_'] = 'Чемпионат России'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат России 2008', 'event_name_'] = 'Чемпионат России'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат России 2009', 'event_name_'] = 'Чемпионат России'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат России 2011', 'event_name_'] = 'Чемпионат России'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат России 2012', 'event_name_'] = 'Чемпионат России'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат России 2013', 'event_name_'] = 'Чемпионат России'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат России 2014', 'event_name_'] = 'Чемпионат России'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат России 2015', 'event_name_'] = 'Чемпионат России'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат России 2016', 'event_name_'] = 'Чемпионат России'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат России 2017', 'event_name_'] = 'Чемпионат России'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат России 2018', 'event_name_'] = 'Чемпионат России'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат России 2019', 'event_name_'] = 'Чемпионат России'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат России 2020', 'event_name_'] = 'Чемпионат России'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Геленджика 2019', 'event_name_'] = 'Чемпионат Геленджика'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Геленджика 2021', 'event_name_'] = 'Чемпионат Геленджика'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Екатеринбурга 2020', 'event_name_'] = 'Чемпионат Екатеринбурга'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Екатеринбурга 2021', 'event_name_'] = 'Чемпионат Екатеринбурга'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Екатеринбурга 2022', 'event_name_'] = 'Чемпионат Екатеринбурга'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Минска 2019', 'event_name_'] = 'Чемпионат Минска'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Москвы 2007', 'event_name_'] = 'Чемпионат Москвы'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Москвы 2008', 'event_name_'] = 'Чемпионат Москвы'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Москвы 2009', 'event_name_'] = 'Чемпионат Москвы'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат России + Будущ.чемпионы', 'event_name_'] = 'Чемпионат России'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Москвы 2011', 'event_name_'] = 'Чемпионат Москвы'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Москвы 2012', 'event_name_'] = 'Чемпионат Москвы'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Москвы 2013', 'event_name_'] = 'Чемпионат Москвы'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Москвы 2014', 'event_name_'] = 'Чемпионат Москвы'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Москвы 2015', 'event_name_'] = 'Чемпионат Москвы'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Москвы 2017', 'event_name_'] = 'Чемпионат Москвы'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Москвы 2018', 'event_name_'] = 'Чемпионат Москвы'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Москвы 2019', 'event_name_'] = 'Чемпионат Москвы'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Москвы 2020', 'event_name_'] = 'Чемпионат Москвы'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Москвы 2021', 'event_name_'] = 'Чемпионат Москвы'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Москвы 2022', 'event_name_'] = 'Чемпионат Москвы'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Поволжья Саратов', 'event_name_'] = 'Чемпионат Поволжья'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат РЦ Самолета', 'event_name_'] = 'Чемпионат РЦ Самолет'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат СПб', 'event_name_'] = 'Чемпионат Санкт-Петербурга'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Санкт-Петербурга 2015', 'event_name_'] = 'Чемпионат Санкт-Петербурга'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Санкт-Петербурга 2016', 'event_name_'] = 'Чемпионат Санкт-Петербурга'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Санкт-Петербурга 2017', 'event_name_'] = 'Чемпионат Санкт-Петербурга'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Санкт-Петербурга 2018', 'event_name_'] = 'Чемпионат Санкт-Петербурга'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Сибири 2015', 'event_name_'] = 'Чемпионат Сибири'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Сибири 2016', 'event_name_'] = 'Чемпионат Сибири'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Сибири 2018', 'event_name_'] = 'Чемпионат Сибири'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Сибири 2021', 'event_name_'] = 'Чемпионат Сибири'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Сибири Красноярск', 'event_name_'] = 'Чемпионат Сибири'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Тюмени 2022', 'event_name_'] = 'Чемпионат Тюмени'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Урала 2017', 'event_name_'] = 'Чемпионат Урала'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Урала 2018', 'event_name_'] = 'Чемпионат Урала'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Урала 2019', 'event_name_'] = 'Чемпионат Урала'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат юга России', 'event_name_'] = 'Чемпионат Юга России'
events_classic.loc[events_classic['event_name_'] == 'Шаг Вперед-2', 'event_name_'] = 'Шаг Вперед'
events_classic.loc[events_classic['event_name_'] == 'Шаг вперед', 'event_name_'] = 'Шаг Вперед'
events_classic.loc[events_classic['event_name_'] == 'Это не ЧР 2020', 'event_name_'] = 'Это не ЧР'
events_classic.loc[events_classic['event_name_'] == 'Это не ЧС - 2020', 'event_name_'] = 'Это не ЧС'
events_classic.loc[events_classic['event_name_'] == 'Январский weekend 2015', 'event_name_'] = 'Январский уикенд'
events_classic.loc[events_classic['event_name_'] == 'Январский ритм 2015', 'event_name_'] = 'Январский ритм'
events_classic.loc[events_classic['event_name_'] == 'Январский ритм 2016', 'event_name_'] = 'Январский ритм'
events_classic.loc[events_classic['event_name_'] == 'Январский ритм 2018', 'event_name_'] = 'Январский ритм'

**Комментарий**:
Формируем информацию о городе проведения турнира

In [527]:
events_classic['event_city'] = events_classic['event_city'].str.lstrip(' ')
events_classic['event_city'] = events_classic['event_city'].str.rstrip(' ')
events_classic.loc[events_classic['event_city'] == 'Екатеринбуг', 'event_city'] = 'Екатеринбург'
events_classic.loc[events_classic['event_city'] == 'Моска', 'event_city'] = 'Москва'
events_classic.loc[events_classic['event_city'] == 'Новосибирск.', 'event_city'] = 'Новосибирск'
events_classic.loc[events_classic['event_city'] == 'Новосибирска', 'event_city'] = 'Новосибирск'
events_classic.loc[events_classic['event_city'] == 'Ростов-На-Дону', 'event_city'] = 'Ростов-на-Дону'
events_classic.loc[events_classic['event_city'] == 'Ростов-на-дону', 'event_city'] = 'Ростов-на-Дону'
events_classic.loc[events_classic['event_city'] == 'Новосибирск.', 'event_city'] = 'Ростов-на-Дону'
events_classic.loc[events_classic['event_city'] == 'Ростова- на-Дону', 'event_city'] = 'Ростов-на-Дону'
events_classic.loc[events_classic['event_city'] == 'СПб', 'event_city'] = 'Санкт-Петербург'
events_classic.loc[events_classic['event_city'] == 'Спб', 'event_city'] = 'Санкт-Петербург'
events_classic.loc[events_classic['event_city'] == 'род', 'event_city'] = 'Москва'
events_classic.loc[events_classic['event_city'] == 'род  г.Москва', 'event_city'] = 'Москва'
events_classic.loc[events_classic['event_city'] == 'род 2018 г.Москва', 'event_city'] = 'Москва'
events_classic.loc[events_classic['event_city'] == 'род г.Москва', 'event_city'] = 'Москва'
events_classic.loc[events_classic['event_city'] == 'рода  г.Москва', 'event_city'] = 'Москва'
events_classic.loc[events_classic['event_city'] == 'рода г.Москва', 'event_city'] = 'Москва'
events_classic.loc[events_classic['event_name_'] == 'Открытый межрегиональный Кубок Сибири', 'event_city'] = 'Красноярск'
events_classic.loc[events_classic['event_name_'] == 'Второй Cибирский межрегиональный конкурс', 'event_city'] = 'Красноярск'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Сибири', 'event_city'] = 'Красноярск'
events_classic.loc[events_classic['event_name_'] == 'Открытый Кубок Новосибирска', 'event_city'] = 'Новосибирск'
events_classic.loc[events_classic['event_name_'] == 'Зимний Кубок Санкт-Петербурга', 'event_city'] = 'Санкт-Петербург'
events_classic.loc[events_classic['event_name_'] == 'Кубок Движения Красноярск', 'event_city'] = 'Красноярск'
events_classic.loc[events_classic['event_name_'] == 'Простые движения', 'event_city'] = 'Санкт-Петербург'
events_classic.loc[events_classic['event_name_'] == 'Кубок Северной Столицы', 'event_city'] = 'Санкт-Петербург'
events_classic.loc[events_classic['event_name_'] == 'Кубок Поволжья', 'event_city'] = 'Саратов'
events_classic.loc[events_classic['event_name_'] == 'Кубок Ренессанса', 'event_city'] = 'Барнаул'
events_classic.loc[events_classic['event_name_'] == 'Кубок Великого Новгорода', 'event_city'] = 'Великий Новгород'
events_classic.loc[events_classic['event_name_'] == 'На пути к звездам', 'event_city'] = 'Красноярск'
#Кубок лета надо дергать по индексам
events_classic.loc[events_classic['event_name_'] == 'Золото Осени', 'event_city'] = 'Санкт-Петербург'
events_classic.loc[events_classic['event_name_'] == 'Кубок ОДС', 'event_city'] = 'Санкт-Петербург'
events_classic.loc[events_classic['event_name_'] == 'Кубок Сибири', 'event_city'] = 'Красноярск'
events_classic.loc[events_classic['event_name_'] == 'Открытый Кубок Новосибирска', 'event_city'] = 'Новосибирск'
events_classic.loc[events_classic['event_name_'] == 'Зимний Кубок Академии Танца', 'event_city'] = 'Саратов'
events_classic.loc[events_classic['event_name_'] == 'Зимний Кубок Санкт-Петербурга', 'event_city'] = 'Санкт-Петербург'
events_classic.loc[events_classic['event_name_'] == 'Кубок Северной Столицы', 'event_city'] = 'Санкт-Петербург'
events_classic.loc[events_classic['event_name_'] == 'Открытый турнир для ED классов', 'event_city'] = 'Красноярск'
events_classic.loc[events_classic['event_name_'] == 'Летний Кубок Академии Танца', 'event_city'] = 'Саратов'
events_classic.loc[events_classic['event_name_'] == 'Чемпионат Юга России', 'event_city'] = 'Красноярск'
events_classic.loc[events_classic['event_name_'] == 'Летний Кубок Новосибирска', 'event_city'] = 'Новосибирск'
events_classic.loc[events_classic['event_name_'] == 'Танцевальный уикенд', 'event_city'] = 'Санкт-Петербург'
events_classic.loc[events_classic['event_name_'] == 'Осенний Кубок Ренессанса', 'event_city'] = 'Барнаул'
events_classic.loc[events_classic['event_name_'] == 'Кубок Морской Славы', 'event_city'] = 'Санкт-Петербург'
events_classic.loc[events_classic['event_name_'] == 'Позитивный Кубок', 'event_city'] = 'Санкт-Петербург'
events_classic.loc[events_classic['event_name_'] == 'Step by Step', 'event_city'] = 'Красноярск'
events_classic.loc[events_classic['event_name_'] == 'Кубок Северо-Запада', 'event_city'] = 'Санкт-Петербург'
events_classic.loc[events_classic['event_name_'] == 'Кубок Движения Красноярск', 'event_city'] = 'Красноярск'
events_classic.loc[events_classic['event_name_'] == 'Кубок Северной Столицы', 'event_city'] = 'Санкт-Петербург'
events_classic.loc[events_classic['event_name_'] == 'Кубок Минска', 'event_city'] = 'Минск'
events_classic.loc[events_classic['event_name_'] == 'Кубок Екатеринбурга', 'event_city'] = 'Екатеринбург'
events_classic.loc[(events_classic['event_name_'] == 'Кубок Лета')&(events_classic['event_city'].isna()), 'event_city'] = 'Санкт-Петербург'
events_classic['event_city'] = events_classic['event_city'].fillna('Москва')

**Комментарий**:
Формируем дату проведения конкурса, а также год и месяц

In [530]:
events_classic['event_date'] = pd.to_datetime(events_classic['event_date'], errors = 'ignore', format='%Y.%m.%d')
events_classic['event_year'] = pd.DatetimeIndex(events_classic['event_date']).year
events_classic['event_month'] = pd.DatetimeIndex(events_classic['event_date']).month

**Комментарий**:
Упорядочим столбцы

In [532]:
events_classic = events_classic[['event_id', 
                                 'event_name_', 
                                 'event_city', 
                                 'event_date', 
                                 'event_year', 
                                 'event_month', 
                                 'nomination',
                                 'count_dancer'
                               ]]

**Комментарий**:
Выделим информацию о гибридных турнирах из информации о турнирах классик

In [533]:
events_classic = events_classic.rename(columns={'event_name_' : 'event_name',
                                                'count_dancer' : 'count_pairs'})

events_hybrid = events_classic[events_classic['count_pairs'].str.contains('/')].reset_index(drop=True)
events_classic = events_classic[~events_classic['count_pairs'].str.contains('/')].reset_index(drop=True)
events_classic = events_classic.loc[~(events_classic['count_pairs'] == 'рейтинг_снят')].reset_index(drop=True)
events_hybrid[['leaders', 'followers']] = events_hybrid['count_pairs'].str.split('/', expand=True)
events_hybrid = events_hybrid.drop(columns = 'count_pairs')

### Информация о турнирах JnJ

**Комментарий**:
Удаление лишних строк и столбцов

In [540]:
events_jnj.drop(events_jnj.columns[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]], axis=1, inplace=True)
events_jnj.columns=events_jnj.loc[0]
events_jnj = events_jnj.iloc[1:7]
events_jnj = events_jnj.T
events_jnj = events_jnj.reset_index(drop=False)
events_jnj['event_info'] = events_jnj[[2, 3, 4, 5, 6]].astype('str').agg(','.join, axis=1) 
events_jnj.drop(events_jnj.columns[[2, 3, 4, 5, 6]], axis=1, inplace=True)

**Комментарий**:
получение и стандартизация информации о номинациях турниров JnJ

In [541]:
events_jnj['event_info'] = events_jnj['event_info'].str.replace(' уч.', '', regex=True)
events_jnj['event_info'] = events_jnj['event_info'].str.replace('-', '', regex=True)
events_jnj['event_info'] = events_jnj['event_info'].str.replace('nan', '', regex=True)
events_jnj['event_info'] = events_jnj['event_info'].str.rstrip(',')
events_jnj['event_info'] = events_jnj['event_info'].str.lstrip(',')
events_jnj['event_info'] = events_jnj['event_info'].str.replace(',,', ',')

events_jnj[[2, 3, 4, 5, 6]] = events_jnj['event_info'].str.split(',', expand=True)
events_jnj[['class_1', 'count_1']] = events_jnj[2].str.split(' ', 1, expand=True)
events_jnj[['class_2', 'count_2']] = events_jnj[3].str.split(' ', 1, expand=True)
events_jnj[['class_3', 'count_3']] = events_jnj[4].str.split(' ', 1, expand=True)
events_jnj[['class_4', 'count_4']] = events_jnj[5].str.split(' ', 1, expand=True)
events_jnj[['class_5', 'count_5']] = events_jnj[6].str.split(' ', 1, expand=True)

events_jnj.drop(events_jnj.columns[[3, 4, 5, 6, 7]], axis=1, inplace=True)
events_jnj = events_jnj.rename(columns={0 : 'event_name', 1 : 'event_date'})
events_jnj = events_jnj.sort_index(ascending=False)
events_jnj = events_jnj.reset_index()
events_jnj = events_jnj.sort_index()
events_jnj = events_jnj.drop(columns = 'index')
events_jnj = events_jnj.reset_index()
events_jnj['index'] = events_jnj['index']+1
events_jnj = events_jnj.rename(columns={'index' : 'event_id_tmp'})
nomination_1 = events_jnj[['event_id_tmp', 'event_name', 'event_date', 'event_info', 'class_1', 'count_1']]
nomination_1 = nomination_1.rename(columns={'class_1' : 'nomination', 'count_1' : 'count_dancer'})
nomination_2 = events_jnj[['event_id_tmp', 'event_name', 'event_date', 'event_info', 'class_2', 'count_2']]
nomination_2= nomination_2.rename(columns={'class_2' : 'nomination', 'count_2' : 'count_dancer'})
nomination_3 = events_jnj[['event_id_tmp', 'event_name', 'event_date', 'event_info', 'class_3', 'count_3']]
nomination_3 = nomination_3.rename(columns={'class_3' : 'nomination', 'count_3' : 'count_dancer'})
nomination_4 = events_jnj[['event_id_tmp', 'event_name', 'event_date', 'event_info', 'class_4', 'count_4']]
nomination_4 = nomination_4.rename(columns={'class_4' : 'nomination', 'count_4' : 'count_dancer'})
nomination_5 = events_jnj[['event_id_tmp', 'event_name', 'event_date', 'event_info', 'class_5', 'count_5']]
nomination_5 = nomination_5.rename(columns={'class_5' : 'nomination', 'count_5' : 'count_dancer'})
events_jnj = pd.concat([nomination_1, nomination_2, nomination_3, nomination_4, nomination_5], axis=0)
events_jnj = events_jnj.sort_values(by='event_id_tmp').fillna(0)
events_jnj = events_jnj.loc[events_jnj['nomination'] != 0]
events_jnj.loc[events_jnj['nomination'] == 'RSS', 'nomination'] = 'RSMS'
events_jnj.loc[events_jnj['nomination'] == 'BRS', 'nomination'] = 'BgRS'
events_jnj.loc[events_jnj['nomination'] == 'B', 'nomination'] = 'Bg'
events_jnj.loc[events_jnj['nomination'] == 'S+Ch', 'nomination'] = 'SCh'
events_jnj.loc[events_jnj['nomination'] == 'BgCh', 'nomination'] = 'BgRSMSCh'
events_jnj.loc[events_jnj['event_id_tmp'] == 57, 'nomination'] = 'Bg'
events_jnj.loc[events_jnj['event_id_tmp'] == 57, 'count_dancer'] = '13/22'

**Комментарий**:
Синхронизируем id турниров с турнирами classic

In [552]:
#напишем функцию, которая берет название клуба и возвращает его id

def search_id_jnj(name):
    for index, row in events_classic_tmp.iterrows():
        if name == row[3]:
            return row[0]
        
        

In [553]:
events_jnj['search'] = events_jnj['event_name'] + ' ' + events_jnj['event_date'].astype('str')

In [555]:
events_jnj.loc[events_jnj['event_name'] =='Чемпионат \nСанкт-\nПетербурга \n2015', 'event_name'] = 'Чемпионат\nСанкт-Петербурга\n2015\nг,СПб'
events_jnj.loc[events_jnj['event_name'] =='Чемпионат \nСанкт-Петербурга \n2016\nг.СПб', 'event_name'] = 'Чемпионат\nСанкт-Петербурга\n2016\nг.СПб'
events_jnj.loc[events_jnj['event_name'] =='Весенний \nКубок \nДвижения\nг. Новосибирск', 'event_name'] = 'Весенний \nКубок \nДвижения\nг.Новосибирск'


In [556]:
#получим информацию об id турниров
events_jnj['event_id'] = events_jnj['search'].apply(search_id_jnj)

In [557]:
#Скорректируем информацию о конкурсах, разные форматы соревнований которых проходили в разные дни
events_jnj.loc[events_jnj['event_name'] == 'Чемпионат \nРоссии \n2014\nг.Москва', 'event_id'] = 267
events_jnj.loc[events_jnj['event_name'] == 'Чемпионат\nСанкт-Петербурга\n2015\nг,СПб', 'event_id'] = 286
events_jnj.loc[events_jnj['event_name'] == 'Чемпионат \nУрала \nг.Екатеринбург', 'event_id'] = 289
events_jnj.loc[events_jnj['event_name'] == 'Чемпионат \nМосквы \n2015\nг.Москва', 'event_id'] = 290
events_jnj.loc[events_jnj['event_name'] == 'HOT WINTER \nCUP \nг.Красноярск', 'event_id'] = 324
events_jnj.loc[events_jnj['event_name'] == 'СуперЛига \nСибири \nг.Новосибирск', 'event_id'] = 340
events_jnj.loc[events_jnj['event_name'] == 'Чемпионат\nСанкт-Петербурга\n2016\nг.СПб', 'event_id'] = 342
events_jnj.loc[events_jnj['event_name'] == 'UralFest 2016 \nг.Екатеринбург', 'event_id'] = 344
events_jnj.loc[events_jnj['event_name'] == 'Весенний \nКубок \nДвижения\nг.Новосибирск', 'event_id'] = 347
events_jnj.loc[events_jnj['event_name'] == 'Открытый\nКубок\n2016\nг.Новосибирск', 'event_id'] = 360
events_jnj.loc[events_jnj['event_name'] == 'Hustle&Discofox\nFestival\nCup\nг.Москва', 'event_id'] = 366
events_jnj.loc[events_jnj['event_name'] == 'Hot Winter\nг.Красноярск', 'event_id'] = 385
events_jnj.loc[events_jnj['event_name'] == 'Чемпионат \nМосквы\n2017\nг.Москва', 'event_id'] = 403
events_jnj.loc[events_jnj['event_name'] == 'Командное\nПервенство\n2017\nг.Москва', 'event_id'] = 405
events_jnj.loc[events_jnj['event_name'] == 'Чемпионат \nРоссии \n2017\nг.Москва', 'event_id'] = 435
events_jnj.loc[events_jnj['event_name'] == 'Dance\nRevolution \nCup 2018 \nг.Москва', 'event_id'] = 445
events_jnj.loc[events_jnj['event_name'] == 'NordCup 2018\nг.СПб', 'event_id'] = 452
events_jnj.loc[events_jnj['event_name'] == 'СуперЛига \nСибири \n2018\nг.Новосибирск', 'event_id'] = 455
events_jnj.loc[events_jnj['event_name'] == 'Чемпионат\nМосквы\n2018\nг,Москва', 'event_id'] = 458
events_jnj.loc[events_jnj['event_name'] == 'HD Cup\nг.Москва', 'event_id'] = 477
events_jnj.loc[events_jnj['event_name'] == 'Танцевальный\nweekend\nг.СПб', 'event_id'] = 481
events_jnj.loc[events_jnj['event_name'] == 'СуперЛига\nСибири\n2019\nг.Новосибирск', 'event_id'] = 503
events_jnj.loc[events_jnj['event_name'] == 'Чемпионат\nУрала\n2019\nг.Екатеринбург', 'event_id'] = 508
events_jnj.loc[events_jnj['event_name'] == 'Командное\nПервенство\nг.Москва', 'event_id'] = 512
events_jnj.loc[events_jnj['event_name'] == 'Танцевальный\nуикенд\nг.СПб', 'event_id'] = 532
events_jnj.loc[events_jnj['event_name'] == 'Чемпионат \nРоссии \n2019\nг.Москва', 'event_id'] = 539
events_jnj.loc[events_jnj['event_name'] == 'Nord Cup\nг.СПб', 'event_id'] = 551
events_jnj.loc[events_jnj['event_name'] == "Rock'n'Dance\nCUP 2020\nг.Красноярск", 'event_id'] = 552
events_jnj.loc[events_jnj['event_name'] == 'XmasCup \n2020\nг.Москва', 'event_id'] = 571
events_jnj.loc[events_jnj['event_name'] == 'Чемпионат\nМосквы\n2021\nг.Москва', 'event_id'] = 589
events_jnj.loc[events_jnj['event_name'] == 'ЧЕМПИОНАТ \nРОССИИ \n2021\nг.Москва', 'event_id'] = 615
events_jnj.loc[events_jnj['event_id_tmp'] == 28, 'event_id'] = 280
events_jnj.loc[events_jnj['event_id_tmp'] == 143, 'event_id'] = 395

**Комментарий**:
Формируем перечень турниров, где не было номинаций classic

In [559]:
events_jnj_only = events_jnj[events_jnj['event_id'].isna()].reset_index(drop=True)
events_jnj = events_jnj[~events_jnj['event_id'].isna()].reset_index(drop=True)

**Комментарий**:
Объединим сведения о турнирах jnj и hybrid

In [562]:
events_jnj = events_jnj[['nomination',	'count_dancer',	'event_id']]

In [563]:
events_jnj = pd.merge(events_jnj, events_classic[['event_id', 'event_name', 'event_city', 'event_date', 'event_year', 'event_month']], how ='inner', on='event_id')

**Комментарий**:
Получим информацию о количестве партнеров и партнерш

In [564]:
events_jnj = events_jnj[['event_id', 'event_name', 'event_city', 'event_date', 'event_year', 'event_month', 'nomination', 'count_dancer']]
events_jnj[['leaders', 'followers']] = events_jnj['count_dancer'].str.split('/', expand=True)
events_jnj = events_jnj.drop(columns='count_dancer')
events_jnj = events_jnj.drop_duplicates().reset_index(drop=True)

**Комментарий**:
Сформируем информацию о турнирах без номинации classic

In [568]:
events_jnj_only['event_name'] = events_jnj_only['event_name'].str.replace('\n', ' ', regex=True)

In [571]:
events_jnj_only[['event_name_', 'event_city']] = events_jnj_only['event_name'].str.split(' г.', 1, expand=True)
events_jnj_only['event_name_'] = events_jnj_only['event_name_'].str.rstrip(' ')
events_jnj_only['event_name_'] = events_jnj_only['event_name_'].str.lstrip(' ')
events_jnj_only['event_name_'] = events_jnj_only['event_name_'].str.replace('  ', ' ', regex=True)

events_jnj_only.loc[events_jnj_only['event_name_'] == 'Jack and Jill Оnly 2021', 'event_name_'] = 'Jack and Jill Оnly'
events_jnj_only.loc[events_jnj_only['event_name_'] == 'Jack and Jill Only', 'event_name_'] = 'Jack and Jill Оnly'
events_jnj_only.loc[events_jnj_only['event_name_'] == 'Jack and Jill Оnly 2022', 'event_name_'] = 'Jack and Jill Оnly'

events_jnj_only['event_date'] = pd.to_datetime(events_jnj_only['event_date'], errors = 'ignore', format='%Y.%m.%d')
events_jnj_only['event_year'] = pd.DatetimeIndex(events_jnj_only['event_date']).year
events_jnj_only['event_month'] = pd.DatetimeIndex(events_jnj_only['event_date']).month

events_jnj_only = events_jnj_only[['event_id', 
                                 'event_name_', 
                                 'event_city', 
                                 'event_date', 
                                 'event_year', 
                                 'event_month', 
                                 'nomination',
                                 'count_dancer'
                               ]]

events_jnj_only = events_jnj_only.rename(columns={'event_name_' : 'event_name',
                                                'count_dancer' : 'count_pairs'})

events_jnj_only[['leaders', 'followers']] = events_jnj_only['count_pairs'].str.split('/', expand=True)
events_jnj_only = events_jnj_only.drop(columns = 'count_pairs')
events_jnj_only['event_id'] = events_jnj_only['event_date'].astype('str').str.replace('-', '', regex=True).astype('int64')


events_jnj = pd.concat([events_jnj, events_jnj_only], axis=0)
events_jnj = events_jnj.sort_values(by='event_date').reset_index(drop=True)
events_jnj.loc[(events_jnj['event_id'] == 391)&(events_jnj['nomination'] == 'M'), 'leaders'] = '2'


events_jnj['leaders'] = events_jnj['leaders'].astype('int64')
events_jnj['followers'] = events_jnj['followers'].astype('int64')
events_classic['count_pairs'] = events_classic['count_pairs'].astype('int64')


## Формируем информацию о результатах турниров

**Комментарий**:
Информация о турнирах Classic

In [581]:
events_classic_res.drop(events_classic_res.columns[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]], axis=1, inplace=True)
events_classic_res = events_classic_res.drop(index = [2, 3, 4, 5, 6, 7, 8, 9, 10])
events_classic_res = events_classic_res.loc[~events_classic_res['Unnamed: 0'].isna()]

In [584]:
events_classic_res.set_index('Unnamed: 0')

,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,...,Unnamed: 667,Unnamed: 668,Unnamed: 669,Unnamed: 670,Unnamed: 671,Unnamed: 672,Unnamed: 673,Unnamed: 674,Unnamed: 675,Unnamed: 676
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
"База конкурсов и баллов АСХ в номинации ""хастл""",Ураган 2023\nг.Москва,XmasFest 2022\nг.Новосибирск,ЧЕМПИОНАТ РОССИИ\n2022\nг.Москва,SNEG-i-RHYTHM\n 2022\nг.Тюмень,Надежды России\n 2022\nг.Москва,МеДвижуха - 2022\nг.Красноярск,ЧЕМПИОНАТ \nСАНКТ-ПЕТЕРБУРГА\nг.СПб,Огни Большого \nГорода 2022\nг.Москва,Upgrade \nFor You \n2022\nг.Владимир,BRING \nTHE ACTION 3\nг.Москва,...,Чемпионат\nРЦ Самолет,Чемпионат\nМосквы 2007,Осенний Бал\nN'Club,Осенний\nКубок\nVestadance,Гран При\nРоссии,Держи Ритм,Майский Бал\nN'Club,Звезды \nХастла,Зимний Бал\nN'Club,"Кубок \nVestaDance \n""Дебют-2"""
Баллы последних конкурсов за JnJ,2023-01-04 00:00:00,2022-12-24 00:00:00,2022-12-17 00:00:00,2022-12-03 00:00:00,2022-12-03 00:00:00,2022-11-27 00:00:00,2022-11-26 00:00:00,2022-11-19 00:00:00,2022-11-05 00:00:00,2022-10-29 00:00:00,...,2007-11-05 00:00:00,2007-11-04 00:00:00,2007-10-28 00:00:00,2007-10-07 00:00:00,2007-09-23 00:00:00,2007-09-09 00:00:00,2007-05-20 00:00:00,2007-03-04 00:00:00,2007-01-28 00:00:00,2007-01-21 00:00:00
10843,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10835,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Х10-12B6/8,NaN,NaN,NaN,NaN,NaN
769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Х10-12B6/8,NaN,NaN,NaN,NaN,NaN
5251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [585]:
events_classic_res = events_classic_res.T

events_classic_res.columns=events_classic_res.loc['Unnamed: 0']
events_classic_res = events_classic_res.reset_index()
events_classic_res = events_classic_res.drop(index = [0])
events_classic_res = events_classic_res.drop(columns = ['index'])

events_classic_res = events_classic_res.rename(columns={'База конкурсов и баллов АСХ в номинации "хастл"': 'event_name',
                                                         'Баллы последних конкурсов за JnJ' : 'event_date'})

events_classic_res = events_classic_res.sort_index(ascending=False)
events_classic_res = events_classic_res.reset_index()
events_classic_res = events_classic_res.sort_index()
events_classic_res = events_classic_res.drop(columns = ['index', 'event_name', 'event_date'])

events_classic_res = events_classic_res.reset_index()
events_classic_res['index'] = events_classic_res['index']+1
events_classic_res = events_classic_res.rename(columns={'index' : 'event_id'})
events_classic_res = events_classic_res.set_index('event_id')
events_classic_res=events_classic_res.T
events_classic_res = events_classic_res.stack().reset_index()
events_classic_res = events_classic_res[events_classic_res['Unnamed: 0'] != 'event_id']
events_classic_res = events_classic_res.reset_index(drop=True)

events_classic_res = events_classic_res.rename(columns={
                                                        'Unnamed: 0' : 'dancer_id',
                                                        'level_1' : 'event_id',
                                                        0 : 'result'
                                                        })

events_classic_res = events_classic_res.query('result != "штраф" \
                                              & result != "изм.порога перехода" \
                                              & result != "аннулирован" \
                                              & result != "анулировано"')
                                              
events_classic_res[['place', 'points', 'zeroing']] = events_classic_res['result'].str.split('/', 2, expand=True)

In [604]:
events_classic_res['place'] = events_classic_res['place'].str.replace('Х10-12', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х10-13', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х11-12', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х11-13', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х13-15', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х13-16', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х14-15', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х15-16', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х15-18', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х16-18', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х17-18', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х17-21', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х19-20', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х19-22', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х20-21', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х20-23', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х21-23', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х22-23', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х23-24', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х24-26', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х24-27', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х24-30', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х25-26', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х27-28', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х27-32', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х27-36', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х28-30', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х29-34', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х33-35', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х6-7', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х6-8', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х7-8', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х8-9', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Д8', 'D8', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('С1', 'C1', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х10', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х14', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х16', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х19', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х23', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х1', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х2', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х3', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х4', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х5', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х6', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х7', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х8', '', regex=True)
events_classic_res['place'] = events_classic_res['place'].str.replace('Х9', '', regex=True)


In [605]:
events_classic_res.loc[events_classic_res['place'] == '(D3)1', 'place'] = 'D1'
events_classic_res.loc[events_classic_res['place'] == '(D3)2', 'place'] = 'D2'
events_classic_res.loc[events_classic_res['place'] == '(D3)3', 'place'] = 'D3'
events_classic_res.loc[events_classic_res['place'] == '(D3)4', 'place'] = 'D4'
events_classic_res.loc[events_classic_res['place'] == '(D3)5', 'place'] = 'D5'
events_classic_res.loc[events_classic_res['place'] == '(D3)6', 'place'] = 'D6'
events_classic_res.loc[events_classic_res['place'] == '(D3)7', 'place'] = 'D7'
events_classic_res.loc[events_classic_res['place'] == '(D3)8', 'place'] = 'D8'
events_classic_res.loc[events_classic_res['place'] == '(D3)9', 'place'] = 'D9'
events_classic_res.loc[events_classic_res['place'] == '(D3)10', 'place'] = 'D10'
events_classic_res.loc[events_classic_res['place'] == '(D3)11', 'place'] = 'D11'
events_classic_res.loc[events_classic_res['place'] == '(D3)12', 'place'] = 'D12'
events_classic_res.loc[events_classic_res['place'] == '(D3)13', 'place'] = 'D13'
events_classic_res.loc[events_classic_res['place'] == '(D3)14', 'place'] = 'D14'
events_classic_res.loc[events_classic_res['place'] == '(D3)15', 'place'] = 'D15'
events_classic_res.loc[events_classic_res['place'] == '(D3)16', 'place'] = 'D16'
events_classic_res.loc[events_classic_res['place'] == '(D3)17', 'place'] = 'D17'
events_classic_res.loc[events_classic_res['place'] == '(D3)18', 'place'] = 'D18'
events_classic_res.loc[events_classic_res['place'] == '(D3)19', 'place'] = 'D19'
events_classic_res.loc[events_classic_res['place'] == '(E3)1', 'place'] = 'E1'

In [606]:
events_classic_res[['place_del', 'place_tru']] = events_classic_res['place'].str.split(')', expand=True)
events_classic_res['place_tru'] = events_classic_res['place_tru'].fillna(events_classic_res['place_del'])
events_classic_res = events_classic_res.drop(columns=['place_del', 'place'])
events_classic_res[['points_del', 'points_tru']] = events_classic_res['points'].str.split('+', expand=True)
events_classic_res['points_tru'] = events_classic_res['points_tru'].fillna(0).astype('int64')
events_classic_res = events_classic_res.drop(columns=['points_del', 'points'])

In [612]:
def get_nomination(cell):
    return list(cell)[0]

def get_place(cell):
    return "".join(list(cell)[1:])

In [614]:
events_classic_res['nomination'] = events_classic_res['place_tru'].apply(get_nomination)
events_classic_res['place'] = events_classic_res['place_tru'].apply(get_place)

In [ ]:
events_classic_res.loc[events_classic_res['nomination'] == '3', 'nomination'] = 'D'
events_classic_res.loc[events_classic_res['dancer_id'] == 7059, 'place'] = '3'
events_classic_res.loc[events_classic_res['nomination'] == 'В', 'nomination'] = 'B'
events_classic_res = events_classic_res.drop(columns=['zeroing', 'place_tru'])

events_classic_res = events_classic_res.rename(columns={'result' : 'result_original',
                                                        'points_tru' : 'rating_points'})

events_classic_res = events_classic_res[['dancer_id', 'event_id', 'result_original', 'nomination', 'place', 'rating_points']]

In [622]:
events_hybrid_res = events_classic_res[events_classic_res['result_original'].str.contains('ДнД')].reset_index(drop=True)

In [623]:
events_classic_res = events_classic_res[~events_classic_res['result_original'].str.contains('ДнД')].reset_index(drop=True)

**Комментарий**:
Информация о турнирах JnJ

In [624]:
events_jnj_res.drop(events_jnj_res.columns[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]], axis=1, inplace=True)
events_jnj_res = events_jnj_res.drop(index = [2, 3, 4, 5, 6, 7, 8, 9, 10])
events_jnj_res = events_jnj_res.loc[~events_jnj_res['Unnamed: 0'].isna()]


In [625]:
events_jnj_res.set_index('Unnamed: 0')
events_jnj_res = events_jnj_res.T
events_jnj_res.columns=events_jnj_res.loc['Unnamed: 0']
events_jnj_res = events_jnj_res.reset_index()
events_jnj_res = events_jnj_res.drop(index = [0])
events_jnj_res = events_jnj_res.drop(columns = ['index'])
events_jnj_res = events_jnj_res.rename(columns={'База конкурсов и баллов АСХ в номинации "хастл-JnJ"': 'event_name',
                                                         'Баллы последних конкурсов за "классику"' : 'event_date'})

events_jnj_res['event_date'] = pd.to_datetime(events_jnj_res['event_date'], errors = 'ignore', format='%Y.%m.%d')
events_jnj_res['search'] = events_jnj_res['event_name'] + ' ' + events_jnj_res['event_date'].astype('str')
events_jnj_res['event_id'] = events_jnj_res['search'].apply(search_id_jnj)

In [629]:
events_jnj_res.loc[events_jnj_res['event_name'] =='Чемпионат \nСанкт-\nПетербурга \n2015', 'event_name'] = 'Чемпионат\nСанкт-Петербурга\n2015\nг,СПб'
events_jnj_res.loc[events_jnj_res['event_name'] =='Чемпионат \nСанкт-Петербурга \n2016\nг.СПб', 'event_name'] = 'Чемпионат\nСанкт-Петербурга\n2016\nг.СПб'
events_jnj_res.loc[events_jnj_res['event_name'] =='Весенний \nКубок \nДвижения\nг. Новосибирск', 'event_name'] = 'Весенний \nКубок \nДвижения\nг.Новосибирск'

In [630]:
events_jnj_res.loc[events_jnj_res['event_name'] == 'Чемпионат \nРоссии \n2014\nг.Москва', 'event_id'] = 267
events_jnj_res.loc[events_jnj_res['event_name'] == 'Чемпионат\nСанкт-Петербурга\n2015\nг,СПб', 'event_id'] = 286
events_jnj_res.loc[events_jnj_res['event_name'] == 'Чемпионат \nУрала \nг.Екатеринбург', 'event_id'] = 289
events_jnj_res.loc[events_jnj_res['event_name'] == 'Чемпионат \nМосквы \n2015\nг.Москва', 'event_id'] = 290
events_jnj_res.loc[events_jnj_res['event_name'] == 'HOT WINTER \nCUP \nг.Красноярск', 'event_id'] = 324
events_jnj_res.loc[events_jnj_res['event_name'] == 'СуперЛига \nСибири \nг.Новосибирск', 'event_id'] = 340
events_jnj_res.loc[events_jnj_res['event_name'] == 'Чемпионат\nСанкт-Петербурга\n2016\nг.СПб', 'event_id'] = 342
events_jnj_res.loc[events_jnj_res['event_name'] == 'UralFest 2016 \nг.Екатеринбург', 'event_id'] = 344
events_jnj_res.loc[events_jnj_res['event_name'] == 'Весенний \nКубок \nДвижения\nг.Новосибирск', 'event_id'] = 347
events_jnj_res.loc[events_jnj_res['event_name'] == 'Открытый\nКубок\n2016\nг.Новосибирск', 'event_id'] = 360
events_jnj_res.loc[events_jnj_res['event_name'] == 'Hustle&Discofox\nFestival\nCup\nг.Москва', 'event_id'] = 366
events_jnj_res.loc[events_jnj_res['event_name'] == 'Hot Winter\nг.Красноярск', 'event_id'] = 385
events_jnj_res.loc[events_jnj_res['event_name'] == 'Чемпионат \nМосквы\n2017\nг.Москва', 'event_id'] = 403
events_jnj_res.loc[events_jnj_res['event_name'] == 'Командное\nПервенство\n2017\nг.Москва', 'event_id'] = 405
events_jnj_res.loc[events_jnj_res['event_name'] == 'Чемпионат \nРоссии \n2017\nг.Москва', 'event_id'] = 435
events_jnj_res.loc[events_jnj_res['event_name'] == 'Dance\nRevolution \nCup 2018 \nг.Москва', 'event_id'] = 445
events_jnj_res.loc[events_jnj_res['event_name'] == 'NordCup 2018\nг.СПб', 'event_id'] = 452
events_jnj_res.loc[events_jnj_res['event_name'] == 'СуперЛига \nСибири \n2018\nг.Новосибирск', 'event_id'] = 455
events_jnj_res.loc[events_jnj_res['event_name'] == 'Чемпионат\nМосквы\n2018\nг,Москва', 'event_id'] = 458
events_jnj_res.loc[events_jnj_res['event_name'] == 'HD Cup\nг.Москва', 'event_id'] = 477
events_jnj_res.loc[events_jnj_res['event_name'] == 'Танцевальный\nweekend\nг.СПб', 'event_id'] = 481
events_jnj_res.loc[events_jnj_res['event_name'] == 'СуперЛига\nСибири\n2019\nг.Новосибирск', 'event_id'] = 503
events_jnj_res.loc[events_jnj_res['event_name'] == 'Чемпионат\nУрала\n2019\nг.Екатеринбург', 'event_id'] = 508
events_jnj_res.loc[events_jnj_res['event_name'] == 'Командное\nПервенство\nг.Москва', 'event_id'] = 512
events_jnj_res.loc[events_jnj_res['event_name'] == 'Танцевальный\nуикенд\nг.СПб', 'event_id'] = 532
events_jnj_res.loc[events_jnj_res['event_name'] == 'Чемпионат \nРоссии \n2019\nг.Москва', 'event_id'] = 539
events_jnj_res.loc[events_jnj_res['event_name'] == 'Nord Cup\nг.СПб', 'event_id'] = 551
events_jnj_res.loc[events_jnj_res['event_name'] == "Rock'n'Dance\nCUP 2020\nг.Красноярск", 'event_id'] = 552
events_jnj_res.loc[events_jnj_res['event_name'] == 'XmasCup \n2020\nг.Москва', 'event_id'] = 571
events_jnj_res.loc[events_jnj_res['event_name'] == 'Чемпионат\nМосквы\n2021\nг.Москва', 'event_id'] = 589
events_jnj_res.loc[events_jnj_res['event_name'] == 'ЧЕМПИОНАТ \nРОССИИ \n2021\nг.Москва', 'event_id'] = 615
events_jnj_res.loc[events_jnj_res['event_date'] == '2015-03-21', 'event_id'] = 280
events_jnj_res.loc[events_jnj_res['event_date'] == '2017-03-18', 'event_id'] = 395

In [632]:
events_jnj_res['event_id'] = events_jnj_res['event_id'].fillna(events_jnj_res['event_date'].astype('str').str.replace('-', '', regex=True).astype('int64'))

In [633]:
events_jnj_res['event_id'] = events_jnj_res['event_id'].astype('int64')

In [634]:
events_jnj_res = events_jnj_res.set_index('event_id')
events_jnj_res = events_jnj_res.drop(columns = ['event_name', 'event_date', 'search'])
events_jnj_res=events_jnj_res.T
events_jnj_res = events_jnj_res.stack().reset_index()
events_jnj_res = events_jnj_res.rename(columns={'Unnamed: 0' : 'dancer_id',
                                                0 : 'result'})
events_jnj_res[['result_tmp', 'result_true']] = events_jnj_res['result'].str.split(' ', expand=True)
events_jnj_res[['place_tmp', 'rating_points']] = events_jnj_res['result_true'].str.split('+', expand=True)
events_jnj_res[['place_full', 'place_false']] = events_jnj_res['place_tmp'].str.split('/', expand=True)
events_jnj_res = events_jnj_res.drop(columns = ['result_tmp', 'result_true', 'place_tmp', 'place_false'])
events_jnj_res['place_full'] = events_jnj_res['place_full'].str.replace('BgG', 'Bg', regex=True)

In [644]:
def get_nomination_jnj(cells):
    for cell in cells:
        if list(cells)[0] in ('M', 'S'):
            return list(cells)[0]
        else:
            return "".join(list(cells)[0:2])

def get_place_jnj(cells):
    for cell in cells:
        if list(cells)[0] in ('M', 'S'):
            return "".join(list(cells)[1:])
        else:
            return "".join(list(cells)[2:])

In [646]:
events_jnj_res['nomination'] = events_jnj_res['place_full'].apply(get_nomination_jnj)

events_jnj_res['place'] = events_jnj_res['place_full'].apply(get_place_jnj)
events_jnj_res = events_jnj_res.drop(columns = 'place_full')
events_jnj_res['rating_points'] = events_jnj_res['rating_points'].fillna('0')

events_jnj_res = events_jnj_res[[ 
                                 'dancer_id',
                                 'event_id',
                                 'result',
                                 'nomination',
                                 'place',
                                 'rating_points'

]]

In [651]:
dancers.dancer_id = dancers.dancer_id.astype('str')
events_classic.event_id = events_classic.event_id.astype('str')
events_classic_res.dancer_id = events_classic_res.dancer_id.astype('str')
events_classic_res.event_id = events_classic_res.event_id.astype('str')
events_classic_res.dancer_id = events_classic_res.dancer_id.astype('str')
events_jnj.event_id = events_jnj.event_id.astype('int').astype('str')
events_jnj_res.dancer_id = events_jnj_res.dancer_id.astype('str')
events_jnj_res.rating_points = events_jnj_res.rating_points.astype('int64')
events_hybrid.event_id = events_hybrid.event_id.astype('str')
events_hybrid.leaders = events_hybrid.leaders.astype('int64')
events_hybrid.followers = events_hybrid.followers.astype('int64')
events_hybrid_res.event_id = events_hybrid_res.event_id.astype('str')
events_hybrid_res.dancer_id = events_hybrid_res.dancer_id.astype('str')
club_dancer.dancer_id = club_dancer.dancer_id.astype('str')
club_dancer.club_id = club_dancer.club_id.astype('str')

In [652]:
dancers_events = pd.concat([events_classic_res[['dancer_id', 'event_id']], events_jnj_res[['dancer_id', 'event_id']]], axis=0)
all_events_date = pd.concat([events_classic[['event_id', 'event_date']], events_jnj[['event_id', 'event_date']]], axis=0)
all_events_date['event_id'] = all_events_date['event_id'].astype('int64').astype('str')
dancers_events['event_id'] = dancers_events['event_id'].astype('int64').astype('str')
all_events_date = all_events_date.drop_duplicates().reset_index(drop=True)
dancers_events = dancers_events.drop_duplicates().reset_index(drop=True)
dancers_events = pd.merge(dancers_events, all_events_date, how ='left', on='event_id')
dancers_events = dancers_events.drop_duplicates().reset_index(drop=True)

In [659]:
clubs.to_excel(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\clubs.xlsx')
dancers.to_excel(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\dancers.xlsx')
events_classic.to_excel(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\events_classic.xlsx')
events_jnj.to_excel(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\events_jnj.xlsx')
events_classic_res.to_excel(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\events_classic_res.xlsx')
events_jnj_res.to_excel(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\events_jnj_res.xlsx')
club_dancer.to_excel(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\club_dancer.xlsx')
events_hybrid.to_excel(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\events_hybrid.xlsx')
events_hybrid_res.to_excel(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\events_hybrid_res.xlsx')
dancers_events.to_excel(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\dancers_events.xlsx')

In [660]:
clubs.to_csv(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\csv\clubs.csv')
dancers.to_csv(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\csv\dancers.csv')
events_classic.to_csv(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\csv\events_classic.csv')
events_jnj.to_csv(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\csv\events_jnj.csv')
events_classic_res.to_csv(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\csv\events_classic_res.csv')
events_jnj_res.to_csv(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\csv\events_jnj_res.csv')
club_dancer.to_csv(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\csv\club_dancer.csv')
events_hybrid.to_csv(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\csv\events_hybrid.csv')
events_hybrid_res.to_csv(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\csv\events_hybrid_res.csv')
dancers_events.to_csv(r'C:\Users\Balak\OneDrive\Документы\GitHub\Pet-project\ASH\Этап 1. Анализ и декомпозиция xlsm файла\work folder\07.01.2023\csv\dancers_events.csv')